# Master Thesis 

Author: Mathieu Leng
Date: December 2023
Version: 1
Goal: This code is related with a master thesis written by Mathieu Leng, under the supervision of Pr. Miguel Nogueira, for Catolica Lisbon School of Business and Economics. 

## 1. Packages

In [ ]:
import pandas as pd  # to handle dataset                                   
import xarray as xr  # to handle dataset from CORDEX and ERA5 sources
import numpy as np   # to handle dataset
import os
import matplotlib.pyplot as plt   # to plot
import re   # to use regular expression
from UTCI.Compute_UTCI import Compute_Tmrt, Compute_UTCI_approx   # code from Pr. Nogeira, to calculate UTCI
import gurobipy as gp  # to do linear programming
from gurobipy import GRB  # to do linear programming
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import seaborn as sns  # to plot
from matplotlib.patches import Patch
import unidecode
from scipy.stats import chi2_contingency
from matplotlib.ticker import FuncFormatter
from xclim import sdba
import unidecode

## 2. Functions

### 2.1 Cleaning Functions

In [ ]:
def custom_round(x):
    """
    Round the value to the quarter
    :param x: a number
    :return: 
    """
    return np.ceil(x * 4) / 4

In [ ]:
def clean_col(df, col_name):
    """
    Cleans the column
    :param df: a pandas dataframe
    :param col_name: name of the columns to clean
    :return: 
        the dataframe
    """
    df[f'{col_name}'] = df[f'{col_name}'].str.replace(',', '.').astype(float)
    df[f'{col_name}'] = df[f'{col_name}'].apply(custom_round)
    return df

In [ ]:
# Function to remove accents from a string
def remove_accents(input_str):
    return unidecode.unidecode(input_str)

### 2.2 file handling Functions

In [ ]:
def get_date(f):
    """
    Creates a date object, getting it from the name of a file
    :param f: file name
    :return: date_obj: date object
    """
    pattern = r'_(\d{8})_'  # regular expression
    match = re.search(pattern, f) # find matching expressions
    date = match.group(1) # get the second element of the matching expression
    date_obj = pd.to_datetime(date, format='%Y%m%d') 
    return date_obj

In [ ]:
def extend_years(df, begin, end):
    """
    extends the Date from begin to end year
    :param df: a pandas dataframe
    :param begin: begin year 
    :param end: end year
    :return: 
        dataframe with extended years
    """
    

    all_years_df = pd.DataFrame()
    
    for year in range(begin, end): # Loop over desired years
        tempo_df = df.copy()
        tempo_df['Date'] = tempo_df['Date'].apply(lambda x: x.replace(year=year))
        all_years_df = pd.concat([all_years_df, tempo_df])

    all_years_df.reset_index(drop=True, inplace=True)
    return all_years_df


In [ ]:

def create_fake_features(df):
    """
    creates fake games for multiple hours 
    """

    new_rows = []
    for index, row in df.iterrows():
        for hour in [12, 15, 18, 21]:
            if row['Hour'] != hour:
                new_row = row.copy()
                new_row['Match Time(CET)'] = pd.Timestamp(row['Date']).replace(hour=hour, minute=0, second=0)
                new_row['Hour'] = hour
                new_rows.append(new_row)

    
    new_rows_df = pd.DataFrame(new_rows)

    df['True_Game'] = 1
    new_rows_df['True_Game'] = 0

    combined_df = pd.concat([df, new_rows_df], ignore_index=True)

    combined_df.sort_values(by='Match Time(CET)', inplace=True)
    combined_df.reset_index(drop=True, inplace=True)
    
    return combined_df

In [ ]:

def get_timeseries_at_lat_lon_time(da: xr.DataArray, target_lat: float, target_lon: float, timestamp: str) -> float:
    """
    Gets the utci value according to the latitude, longitude, and timestamp
    :param da: Data Array
    :param target_lat: value of the target latitude 
    :param target_lon: value of the target latitude 
    :param timestamp: timestamp
    :return: selected_data: 
    """
    variable_da = da['utci']
    timestamp = pd.to_datetime(timestamp) # Convert timestamp to pandas datetime to match the format in da

    # Get the nearest latitude and longitude
    nearest_lat = variable_da.sel(lat=target_lat, method='nearest').lat.values
    nearest_lon = variable_da.sel(lon=target_lon, method='nearest').lon.values

    # Select data based on nearest latitude, longitude, and exact timestamp
    selected_data = variable_da.sel(lat=nearest_lat, lon=nearest_lon,time=timestamp,method='nearest').values
    
    return selected_data
    


In [ ]:
def get_timeseries_at_rlat_rlon_time(da: xr.DataArray,target_lat: float,target_lon: float, timestamp: np.datetime64) -> xr.DataArray:
    """
    return the value from da, having corresponding timestamp, latitude, and longitude
    :param da: the to-be merged dataset
    :param target_lat: the target latitude
    :param target_lon: the target longitude
    :param timestamp: timestamp
    :return: 
        res: corresponding value
    """
    sliced_ds=da.sel(time=timestamp,method='nearest') # slice for only the correct date
    i,j=np.unravel_index(np.argmin(np.sqrt( (sliced_ds['lat'].values-target_lat)**2+ (sliced_ds['lon'].values-target_lon)**2)),da['lat'].shape) # calculates row and col  index to the closest data from target lat and long
    res = float(sliced_ds.values[i,j])
    return res


def get_var_for_row(row, da):
    """
    calls another function, sending the right parameters
    :param row: line in the original dataset
    :param da: the to-be merged dataset
    :return: 
    """
    return get_timeseries_at_rlat_rlon_time(da, row['latitude'], row['longitude'], row['timestamp'])



In [ ]:
def fill_1_year(folders_path, result):
    """
    fill one year of data
    :param folders_path: path to the folder
    :param result: final dataframe will all the data
    :return: 
        result: final dataframe will all the data
    """
    nc_files = [f for f in os.listdir(folders_path) if f.endswith('.nc')]
    for file in nc_files: #  each file is one year
        with xr.open_dataset(os.path.join(folders_path, file)) as ds:
            variable = list(ds.variables)[-1]  # each file is adifferent varibale
            year = list(ds['time'].dt.year.values)[0] # get the year of this file
            result.loc[result["year"] == year,f'{variable}'] = result[result["year"] == year].apply(get_var_for_row, da=ds[f'{variable}'], axis=1)
    return result

        

### 2.3 Calculations

In [ ]:
def calculate_UTCI(df):
    """
    Calculates utci value
    :param df: pandas dataframe
    :return: 
        the pandas dataframe with UTCI values
    """
    df['tas']=df['tas']-273.15
    df['rlus']=df['rsds']-df['rsus']+df['rlds']
    Tmrt=Compute_Tmrt(df['rlus'],df['rlds'],df['rsus'],df['rsds'])
    Tmrt=Tmrt-273.15
    UTCI=Compute_UTCI_approx(df['tas'],df['hurs'],Tmrt,df['sfcWind'])
    df["UTCI"] = UTCI
    return df

In [ ]:
def categorize_stress(df):
    """
    Add a new column Stress Category based on the UTCI heat stress categories
    :param df: pandas dataframe
    :return: 
        Dataframe with new column Stress Category
    """
    conditions = [
        (df['utci'] > 46),
        (df['utci'] >= 38) & (df['utci'] <= 46),
        (df['utci'] >= 32) & (df['utci'] < 38),
        (df['utci'] >= 26) & (df['utci'] < 32),
        (df['utci'] >= 9) & (df['utci'] < 26),
        (df['utci'] >= 0) & (df['utci'] < 9),
        (df['utci'] >= -13) & (df['utci'] < 0),
        (df['utci'] >= -27) & (df['utci'] < -13),
        (df['utci'] >= -40) & (df['utci'] < -27),
        (df['utci'] < -40)
    ]

    choices = [
        'Extreme heat stress',
        'Very strong heat stress',
        'Strong heat stress',
        'Moderate heat stress',
        'No thermal stress',
        'Slight cold stress',
        'Moderate cold stress',
        'Strong cold stress',
        'Very strong cold stress',
        'Extreme cold stress'
    ]

    df['Stress Category'] = np.select(conditions, choices, default=np.nan)
    return df



In [ ]:
def assign_authorities_conditions(utci):
    """
    Matches policies and UTCI values
    :param utci: UTCI value
    :return: 
        the policy matching the UTCI value
    """
    if utci >= 46: # approximation bc  WBGT 32 not in the table
        return "FIFA cooling break mandatory"
    elif utci >= 38: # 28 WBGT is 38 UTCI, limitation because not exactly like that in the table 
        return "FIFPro game rescheduled"
    elif utci >= 35.8: # ± 26 WBGT
        return "FIFPro cooling breaks mandatory"
    else:
        return "No policies"
    

### 2.4 Plotting

In [ ]:
def silhouette_score_clusters(df, range_k):
    """
    Calculate the silhouette score 
    :param df: pandas dataframe
    :param range_k: range of k 
    :return: 
    """
    # Grouping the data by location and calculating the median and standard deviation of UTCI values
    location_utci_stats = df.groupby('Home Team')['utci'].agg(['median', 'std']).reset_index()

    # Normalizing the median and standard deviation of UTCI values
    scaler = StandardScaler()
    location_utci_stats[['median_normalized', 'std_normalized']] = scaler.fit_transform(
        location_utci_stats[['median', 'std']])
    
    print(location_utci_stats[['median_normalized', 'std_normalized']].isna().any())
    # Handling NaN values
    location_utci_stats[['median_normalized', 'std_normalized']] = location_utci_stats[
        ['median_normalized', 'std_normalized']].fillna(0)

    # Finding the optimal number of clusters using silhouette scores
    silhouette_scores = []

    for k in range_k:
        kmeans = KMeans(n_clusters=k, random_state=0)
        cluster_labels = kmeans.fit_predict(location_utci_stats[['median_normalized', 'std_normalized']])
        silhouette_avg = silhouette_score(location_utci_stats[['median_normalized', 'std_normalized']], cluster_labels)
        silhouette_scores.append(silhouette_avg)

    plt.figure(figsize=(10, 6))
    plt.plot(range_k, silhouette_scores, marker='o')
    plt.title('Silhouette Scores for Different Numbers of Clusters')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Silhouette Score')
    plt.xticks(range_k)
    plt.grid(False)  # Disable the grid
    plt.gca().set_facecolor('white')  # Set the background color to white

    # Save the plot
    silhouette_file_path = 'plots/silhouette_plot.png'
    plt.savefig(silhouette_file_path, bbox_inches='tight')  # Save with tight bounding box

    # Display the silhouette scores
    return silhouette_scores, location_utci_stats


In [ ]:
def optimize_clusters(df):
    """
    Calculate the optimized clusters
    :param df: pandas dataframe
    :return: 
    """
    
    range_k = range(2, 11)  # Testing for 2 to 10 clusters
    silhouette_scores, location_utci_stats = silhouette_score_clusters(df, range_k)

    # Choosing the best K based on the highest silhouette score
    best_k = range_k[np.argmax(silhouette_scores)]

    # Clustering with the optimal number of clusters
    kmeans_optimal = KMeans(n_clusters=best_k, random_state=0)
    location_utci_stats['Optimal_Cluster'] = kmeans_optimal.fit_predict(location_utci_stats[['median_normalized', 'std_normalized']])

    # Plotting the results with the optimal number of clusters
    plt.figure(figsize=(15, 8))
    sns.scatterplot(x=location_utci_stats['Home Team'], y=location_utci_stats['median'],
                    size=location_utci_stats['std'], hue=location_utci_stats['Optimal_Cluster'], palette='Set1')
    plt.xticks(rotation=90)
    plt.title(f'K-Means Clustering of Locations Based on UTCI Median and Standard Deviation ({best_k} Clusters)')
    plt.xlabel('Home Team')
    plt.ylabel('Median UTCI Value')
    plt.legend(title='Optimal Cluster')

    plt.show()
    return location_utci_stats

In [ ]:
def plot_locations_histogram(df):
    """
    Plots the number of games per location 
    :param df: 
    :return: 
    """
    # Count the frequency of each category
    category_counts = df['Home Team'].value_counts()

    # Plot the histogram
    plt.figure(figsize=(10, 8))
    category_counts.plot(kind='bar')
    plt.title('Histogram of Number of Features by Teams')
    plt.xlabel('Locations')
    plt.ylabel('Number of Games')
    plt.xticks(rotation=45, ha='right')  # Rotate labels for better readability
    plt.tight_layout()  # Adjust layout for better fit
    
    # Remove the grid and set the background color to white
    plt.grid(False)
    plt.gca().set_facecolor('white')
    
    plt.savefig("plots/games_by_locations.png", )
    plt.show()


In [ ]:
def plot_overall_percentage_stress_category(df, title, stress_category = 'Stress Category'):
    """
    Plots the percentage per heat stress category
    :param df: 
    :param title: 
    :return: 
    """
    # Calculate the percentage of each stress category
    stress_category_percentages = df[stress_category].value_counts(normalize=True) * 100

    # Color mapping for the stress categories
    colors = {'No thermal stress': 'green', 'Moderate heat stress': 'orange',
              'Strong heat stress': 'red', 'Very strong heat stress': 'darkred',
              'Extreme heat stress': 'black', 'Slight cold stress': 'lightblue'}

    # Ordering the stress categories for consistent color mapping
    ordered_categories = stress_category_percentages.index.map(colors)

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))
    stress_category_percentages.plot(kind='bar', ax=ax, color=ordered_categories)
    fig.patch.set_facecolor('white')  # Set the background color of the figure to white
    ax.set_facecolor('white')  # Set the background color of the axes to white
    

    # Removing grid
    ax.grid(False)

    # Adding percentage annotations on each bar
    for p in ax.patches:
        ax.annotate(f"{p.get_height():.1f}%", (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 10), textcoords='offset points')

    # Adding labels and title
    ax.set_ylabel('Percentage')
    ax.set_xlabel('Stress Category')
    ax.set_title(title)

    # Save the plot to a file
    plt.tight_layout()
    plt.savefig(f'plots_story/{title}.png')

    # Show the plot
    plt.show()


In [ ]:
def plot_overall_percentage_policies_category(df, title, policies = 'policies'):
    """
    Plots the percentage per policy category
    :param df: 
    :param title: 
    :return: 
    """
    # Calculate the percentage of each stress category
    stress_category_percentages = df[policies].value_counts(normalize=True) * 100

    # Color mapping for the stress categories
    colors = {"No policies": 'green', "FIFPro cooling breaks mandatory": 'orange',
              "FIFPro game rescheduled": 'red', "FIFA cooling break mandatory": 'darkred'}

    # Ordering the stress categories for consistent color mapping
    ordered_categories = stress_category_percentages.index.map(colors)

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))
    stress_category_percentages.plot(kind='bar', ax=ax, color=ordered_categories)
    fig.patch.set_facecolor('white')  # Set the background color of the figure to white
    ax.set_facecolor('white')  # Set the background color of the axes to white


    # Removing grid
    ax.grid(False)

    # Adding percentage annotations on each bar
    for p in ax.patches:
        ax.annotate(f"{p.get_height():.1f}%", (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 10), textcoords='offset points')

    # Adding labels and title
    ax.set_ylabel('Percentage')
    ax.set_xlabel('Policies Category')
    ax.set_title(title)

    # Save the plot to a file
    plt.tight_layout()
    plt.savefig(f'plots_story/{title}.png')

    # Show the plot
    plt.show()


In [ ]:
def plot_optimal_cluster_percentage_stress_category_histogram(df, title):
    grouped_data = df.groupby(['Optimal_Cluster', 'Stress Category']).size().unstack().fillna(0)

    # Convert counts to percentages
    grouped_percentage = grouped_data.div(grouped_data.sum(axis=1), axis=0) * 100

    # Define the desired order of stress categories
    order = ['No thermal stress', 'Moderate heat stress', 'Strong heat stress',
             'Very strong heat stress', 'Slight cold stress']

    # Reorder the columns of grouped_percentage according to the defined order
    grouped_percentage = grouped_percentage[order]

    # Color mapping for the stress categories
    colors = {'No thermal stress': 'green', 'Moderate heat stress': 'orange',
              'Strong heat stress': 'red', 'Very strong heat stress': 'darkred',
              'Extreme heat stress': 'black', 'Slight cold stress': 'lightblue',
              'Moderate cold stress': 'blue'}

    # Ordering the stress categories for consistent color mapping
    ordered_categories = [colors[category] for category in order]

    # Plot the histogram
    fig, ax = plt.subplots(figsize=(12, 8))
    fig.patch.set_facecolor('white')  # Set the background color of the figure to white
    ax.set_facecolor('white')  # Set the background color of the axes to white
    grouped_percentage.plot(kind='bar', stacked=True, color=ordered_categories, ax=ax)

    # Adding percentage annotations on each segment of the stacked bars
    for bars in ax.containers:
        ax.bar_label(bars, fmt='%.1f%%', label_type='center')

    # Remove the grid
    ax.grid(False)

    plt.title(title)
    plt.xlabel('Optimal Cluster')
    plt.ylabel('Percentage')
    plt.xticks(rotation=45, ha='right')  # Rotate labels for better readability
    plt.legend(title='Stress Category', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()  # Adjust layout for better fit
    # Remove the spines (the rectangle around the plot)
    for spine in plt.gca().spines.values():
        spine.set_visible(False)
    # Save the plot to a file
    plt.savefig(f'plots_story/{title}.png', facecolor=fig.get_facecolor(), edgecolor='none')

    # Show the plot
    plt.show()


In [ ]:
def plot_grouped_percentage_stress_category_histogram(df):
    # Group the data by 'Hour' and then by 'Stress Category'
    grouped_data = df.groupby(['Hour', 'Stress Category']).size().unstack().fillna(0)

    # Convert counts to percentages
    grouped_percentage = grouped_data.div(grouped_data.sum(axis=1), axis=0) * 100

    # Define the desired order of stress categories
    order = ['No thermal stress', 'Moderate heat stress', 'Strong heat stress',
             'Very strong heat stress', 'Extreme heat stress', 'Slight cold stress']

    # Reorder the columns of grouped_percentage according to the defined order
    grouped_percentage = grouped_percentage[order]

    # Color mapping for the stress categories
    colors = {'No thermal stress': 'green', 'Moderate heat stress': 'orange',
              'Strong heat stress': 'red', 'Very strong heat stress': 'darkred',
              'Extreme heat stress': 'black', 'Slight cold stress': 'lightblue',
              'Moderate cold stress': 'blue'}

    # Ordering the stress categories for consistent color mapping
    ordered_categories = [colors[category] for category in order]

    # Plot the histogram
    fig, ax = plt.subplots(figsize=(12, 8))
    grouped_percentage.plot(kind='bar', stacked=True, color=ordered_categories, ax=ax)

    # Set background color to white
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    # Remove the grid
    ax.grid(False)

    # Adding percentage annotations on each segment of the stacked bars
    for bars in ax.containers:
        ax.bar_label(bars, fmt='%.1f%%', label_type='center')

    # Set titles and labels
    plt.title('Percentage Histogram of Stress Categories Grouped by Hour')
    plt.xlabel('Hour')
    plt.ylabel('Percentage')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Stress Category', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()

    # Remove the spines (the rectangle around the plot)
    for spine in plt.gca().spines.values():
        spine.set_visible(False)

    # Save the plot to a file
    plt.savefig('plots_story/grouped_stress_category_histogram_by_hour.png', facecolor=fig.get_facecolor(), edgecolor='none')

    # Show the plot
    plt.show()


In [ ]:
def plot_grouped_month_percentage_stress_category_histogram(df):
    # Group the data by 'Month' and then by 'Stress Category'
    grouped_data = df.groupby(['month', 'Stress Category']).size().unstack().fillna(0)

    # Convert counts to percentages
    grouped_percentage = grouped_data.div(grouped_data.sum(axis=1), axis=0) * 100

    # Define the desired order of stress categories
    order = ['No thermal stress', 'Moderate heat stress', 'Strong heat stress',
             'Very strong heat stress', 'Extreme heat stress', 'Slight cold stress']

    # Reorder the columns of grouped_percentage according to the defined order
    grouped_percentage = grouped_percentage[order]

    # Color mapping for the stress categories
    colors = {'No thermal stress': 'green', 'Moderate heat stress': 'orange',
              'Strong heat stress': 'red', 'Very strong heat stress': 'darkred',
              'Extreme heat stress': 'black', 'Slight cold stress': 'lightblue',
              'Moderate cold stress': 'blue'}

    # Ordering the stress categories for consistent color mapping
    ordered_categories = [colors[category] for category in order]

    # Plot the histogram
    fig, ax = plt.subplots(figsize=(12, 8))
    grouped_percentage.plot(kind='bar', stacked=True, color=ordered_categories, ax=ax)

    # Set background color to white, remove grid and spines
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    ax.grid(False)

    # Remove the spines (the rectangle around the plot)
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Adding percentage annotations on each segment of the stacked bars
    for bars in ax.containers:
        ax.bar_label(bars, fmt='%.1f%%', label_type='center')

    # Set titles and labels
    plt.title('Percentage Histogram of Stress Categories Grouped by Month')
    plt.xlabel('Month')
    plt.ylabel('Percentage')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Stress Category', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()

    # Save the plot to a file
    plt.savefig('plots_story/grouped_stress_category_histogram_by_month.png', facecolor=fig.get_facecolor(), edgecolor='none')

    # Show the plot
    plt.show()


In [ ]:
def plot_stress_category_percentage(df):
    """
    Plot the percentage of stress categories over years for each home team in the dataframe.
    
    Parameters:
    df (DataFrame): The input dataframe containing the stress category data.
    """

    # Group the original data by 'Home Team', 'year', and 'Stress Category'
    category_counts_per_team_year = df.groupby(['Home Team', 'year', 'Stress Category']).size().reset_index(name='count')
    total_counts_per_team_year = df.groupby(['Home Team', 'year']).size().reset_index(name='total_count')

    # Merge and calculate percentages
    category_percentage_per_team = category_counts_per_team_year.merge(total_counts_per_team_year, on=['Home Team', 'year'])
    category_percentage_per_team['percentage'] = (category_percentage_per_team['count'] / category_percentage_per_team['total_count']) * 100

    # Get a list of unique Home Teams
    home_teams = category_percentage_per_team['Home Team'].unique()

    # Plot settings
    colors = {
        'Very strong heat stress': 'darkred',
        'Strong heat stress': 'red',
        'No thermal stress': 'darkgreen'  # Setting the color for 'No thermal stress'
    }

    # Plot the line plots for each Home Team
    for team in home_teams:
        team_data = category_percentage_per_team[category_percentage_per_team['Home Team'] == team]
        pivot_data = team_data.pivot(index='year', columns='Stress Category', values='percentage')
        pivot_data = pivot_data.fillna(0)  # Fill NaN values with 0

        # Plotting
        ax = pivot_data.plot(kind='line', marker='o', title=f'Stress Category Percentage Over Years for {team}',
                             color=[colors.get(x, '#1f77b4') for x in pivot_data.columns])
        ax.set_ylabel('Percentage')
        ax.set_xlabel('Year')
        ax.grid(True)
        ax.legend(title='Stress Category', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()




In [ ]:
def plot_stress_category_distribution_by_hour(dataframe, utci_column, stress_column, title):
    """
    Plot a bar chart with custom colors and values showing the distribution of stress categories for a given UTCI category across hours.

    :param dataframe: DataFrame containing the data.
    :param utci_column: Column name for the UTCI category.
    :param stress_column: Column name for the stress category.
    :param title: Title for the plot.
    """
    # Custom colors for stress categories
    colors = {'No thermal stress': 'green', 'Moderate heat stress': 'orange',
              'Strong heat stress': 'red', 'Very strong heat stress': 'darkred',
              'Extreme heat stress': 'black', 'Slight cold stress': 'lightblue'}

    # Filter the dataset for the specific UTCI category and group by hour and stress category
    hour_stress_group = dataframe.groupby(['hour', stress_column]).size().unstack(fill_value=0)

    # Reorder the columns to include all possible categories
    all_categories = ['No thermal stress', 'Moderate heat stress', 'Strong heat stress',
                      'Very strong heat stress', 'Extreme heat stress', 'Slight cold stress']
    hour_stress_group = hour_stress_group.reindex(columns=all_categories, fill_value=0)

    # Calculate percentages instead of counts
    hour_stress_percent = hour_stress_group.div(hour_stress_group.sum(axis=1), axis=0) * 100

    # Plotting
    ax = hour_stress_percent.plot(kind='bar', stacked=True, figsize=(12, 8),
                                  color=[colors.get(x, '#333333') for x in all_categories])

    plt.title(title)
    plt.xlabel('Hour')
    plt.ylabel('Percentage')
    plt.legend(title='Stress Category')
    ax.set_facecolor('white')  # Set the background color to white
    plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0f}%'.format(y))) # Format y-axis labels as percentages

    # Adding values on top of each bar
    for p in ax.patches:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy()
        if height > 0: # To avoid displaying 0%
            ax.annotate(f'{height:.0f}%', (x + width/2, y + height/2), ha='center')
    for spine in ax.spines.values():
        spine.set_visible(False)

    plt.savefig(f"plots/{title}.pdf")

    plt.show()




In [ ]:
def plot_stress_category_distribution_with_values(dataframe, stress_column, title):
    """
    Plot a bar chart with custom colors and values showing the distribution of stress categories for a given UTCI category across months.

    :param dataframe: DataFrame containing the data.
    :param stress_column: Column name for the stress category.
    :param title: Title for the plot.
    """
    # Custom colors for stress categories
    colors = {'No thermal stress': 'green', 'Moderate heat stress': 'orange',
              'Strong heat stress': 'red', 'Very strong heat stress': 'darkred',
              'Extreme heat stress': 'black', 'Slight cold stress': 'lightblue'}

    # Filter the dataset for the specific UTCI category and group by month and stress category
    month_stress_group = dataframe.groupby(['month', stress_column]).size().unstack(fill_value=0)

    # Reorder the columns to include all possible categories
    all_categories = ['No thermal stress', 'Moderate heat stress', 'Strong heat stress',
                      'Very strong heat stress', 'Extreme heat stress', 'Slight cold stress']
    month_stress_group = month_stress_group.reindex(columns=all_categories, fill_value=0)

    # Calculate percentages instead of counts
    month_stress_percent = month_stress_group.div(month_stress_group.sum(axis=1), axis=0) * 100

    # Plotting
    ax = month_stress_percent.plot(kind='bar', stacked=True, figsize=(12, 8),
                                   color=[colors.get(x, '#333333') for x in all_categories])

    plt.title(title)
    plt.xlabel('Month')
    plt.ylabel('Percentage')
    plt.legend(title='Stress Category')
    ax.set_facecolor('white')  # Set the background color to white
    plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0f}%'.format(y))) # Format y-axis labels as percentages

    # Adding values on top of each bar
    for p in ax.patches:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy()
        if height > 0: # To avoid displaying 0%
            ax.annotate(f'{height:.0f}%', (x + width/2, y + height/2), ha='center')

    for spine in ax.spines.values():
        spine.set_visible(False)
    
    plt.savefig(f"plots/{title}.pdf")
    plt.show()




In [ ]:
def categorize_stress_future(df, col, stress_cat):
    conditions = [
        (df[f'{col}'] > 46),
        (df[f'{col}'] >= 38) & (df[f'{col}'] <= 46),
        (df[f'{col}'] >= 32) & (df[f'{col}'] < 38),
        (df[f'{col}'] >= 26) & (df[f'{col}'] < 32),
        (df[f'{col}'] >= 9) & (df[f'{col}'] < 26),
        (df[f'{col}'] >= 0) & (df[f'{col}'] < 9),
        (df[f'{col}'] >= -13) & (df[f'{col}'] < 0),
        (df[f'{col}'] >= -27) & (df[f'{col}'] < -13),
        (df[f'{col}'] >= -40) & (df[f'{col}'] < -27),
        (df[f'{col}'] < -40)
    ]

    choices = [
        'Extreme heat stress',
        'Very strong heat stress',
        'Strong heat stress',
        'Moderate heat stress',
        'No thermal stress',
        'Slight cold stress',
        'Moderate cold stress',
        'Strong cold stress',
        'Very strong cold stress',
        'Extreme cold stress'
    ]

    df[f'{stress_cat}'] = np.select(conditions, choices, default=np.nan)
    return df

In [ ]:
# Updated function to plot stress categories on the x-axis with specified color legend for each stress type
def plot_stress_category_distribution_colored(df, stress_columns):
    """
    Plots a bar chart of the percentage of each stress type within each stress category with specified colors.

    :param df: DataFrame containing the data.
    :param stress_columns: List of column names containing stress category data.
    """
    # Define the color mapping for each stress type
    color_mapping = {
        'No thermal stress': 'green',
        'Slight cold stress': 'aqua',
        'Moderate heat stress': 'darkorange',
        'Moderate cold stress': 'deepskyblue',
        'Strong heat stress': 'red',
        'Very strong heat stress': 'maroon',
        'Extreme heat stress': 'purple'
    }

    # Initialize an empty DataFrame to store percentage values
    stress_percentages = pd.DataFrame()

    # Calculate the percentage of each stress type for each stress category
    for column in stress_columns:
        stress_counts = df[column].value_counts(normalize=True) * 100
        stress_percentages[column] = stress_counts

    # Transpose the DataFrame for plotting
    stress_percentages = stress_percentages.T.fillna(0)

    # Prepare a list of colors based on the stress types
    colors = [color_mapping.get(x, 'grey') for x in stress_percentages.columns]

    # Plot
    ax = stress_percentages.plot(kind='bar', stacked=True, color=colors, figsize=(14, 7))

    # Annotate the percentages on the bars
    for c in ax.containers:
        labels = [f'{v.get_height():.1f}%' if v.get_height() > 0 else '' for v in c]
        ax.bar_label(c, labels=labels, label_type='center')

    legend_patches = [Patch(color=color, label=label) for label, color in color_mapping.items()]
    plt.legend(handles=legend_patches, title='Stress Types')

    plt.title('Percentage of each Stress Type by Stress Category')
    plt.xlabel('Stress Category')
    plt.ylabel('Percentage')
    plt.xticks(rotation=45)
    plt.tight_layout()  # Adjust the plot to ensure everything fits without overlapping

    # Remove the grid and set the background color to white
    plt.grid(False)
    plt.gca().set_facecolor('white')
    # Remove the spines (the rectangle around the plot)
    for spine in plt.gca().spines.values():
        spine.set_visible(False)
    plt.savefig("plots/percentage_different_ssp.pdf", format='pdf', bbox_inches='tight')    




In [ ]:
def plot_and_save_stress_category_percentage_ordered(df, rcp_column, rcp_value, save_path, color_mapping, order):
    """
    Plots and saves the stress category percentage bar chart for a given RCP with specified colors and order.

    :param df: DataFrame containing the dataset.
    :param rcp_column: The column name representing the stress category for the specific RCP.
    :param rcp_value: The RCP value (e.g., '2.6', '4.5', '8.5') used in the plot title.
    :param save_path: Path where the plot image will be saved.
    :param color_mapping: Dictionary mapping stress categories to colors.
    :param order: The order in which to display the stress categories.
    """
    # Group by Optimal_Cluster and Stress Category, then count
    counts = df.groupby(['Optimal_Cluster', rcp_column]).size().unstack(fill_value=0)

    # Reorder the columns according to the specified order
    counts = counts[order]

    # Calculate percentages
    percentages = counts.div(counts.sum(axis=1), axis=0) * 100

    # Colors for each category in the specified order
    category_colors = [color_mapping.get(x, 'gray') for x in order]

    # Plotting
    ax = percentages.plot(kind='bar', stacked=True, color=category_colors, figsize=(10, 6), legend=True)
    plt.title(f'Stress Category Percentages for RCP {rcp_value}')
    plt.ylabel('Percentage')
    plt.xlabel('Optimal_Cluster')
    plt.grid(False)
    ax.set_facecolor('white')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

    # Annotating percentages on the bars
    for n, x in enumerate([*percentages.index.values]):
        for (proportion, y_loc) in zip(percentages.loc[x],
                                       percentages.loc[x].cumsum()):
            plt.text(x=n, y=(y_loc - proportion) + (proportion / 2),
                     s=f'{proportion:.1f}%', ha='center', va='center', fontsize=8)

    # Save the plot
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()

color_mapping = {
    'No thermal stress': 'green',
    'Slight cold stress': 'aqua',
    'Moderate heat stress': 'darkorange',
    'Moderate cold stress': 'deepskyblue',
    'Strong heat stress': 'red',
    'Very strong heat stress': 'maroon',
    'Extreme heat stress': 'purple'
}




## 3. Fill Datasets

In [ ]:
path = "your_path"

### 3.1 Locations of the Stadiums

#### 3.1.1 Import

In [ ]:
fname='football_stadiums_2023.csv'
df = pd.read_csv(path+fname, delimiter=";")
df.head()

#### 3.1.2 Cleaning

In [ ]:
df.drop(["Division", "Stadium"], axis = "columns", inplace = True)

In [ ]:
for col_name in ["Longitude", "Latitude"]:
    df = clean_col(df, col_name)
    
df = df.rename(columns={
    'Longitude': 'longitude',
    'Latitude': 'latitude'
})

In [ ]:
df = df.astype({ 'Team' : str})

In [ ]:
df.head()

In [ ]:
df.to_csv('datasets/laliga_rounded.csv')

### 3.2 La Liga fixtures

#### 3.2.1 Import

In [ ]:
fname='La_Liga_Interactive_Table_2022-23_20230521.xlsm'
df_liga = pd.read_excel(path+fname, sheet_name='fixtures')
df_liga.head()

#### 3.2.2 Cleaning and Filtering

In [ ]:
df_liga = df_liga[['Match Time(CET)', 'Home Team']]

In [ ]:
df_liga = df_liga.dropna(how='all') # remove NA, a few last lines were empty

In [ ]:
replacements = {'Real Sociedad':'Real Sociedad ', 'Elche CF': 'Elche Club de Futbol', 'RCD Mallorca': 'Majorque',  'RCD Espanyol' : 'Real Club Deportivo Espanyol','Getafe CF':'Getafe', 'UD Almería' :'Almeria','Cádiz CF': "Cadix", "CA Osasuna" : "Osasuna", 'Real Betis': 'Betis Seville', "Girona FC" : "Gérone", "FC Barcelona" : "FC Barcelone", "RC Celta" : "Celta Vigo", "Villarreal CF" : "Villareal", "Athletic Club": "Athletic Bilbao", "Valencia CF": "Valence FC", "Real Valladolid CF" : "Real Valladolid Club de Futbol", "Sevilla FC" : "Séville FC", "Huesca" : "Sociedad Deportiva Huesca" }

df_liga['Home Team'] = df_liga['Home Team'].replace(replacements) # replace the name to have matching names between the 2 datasets

In [ ]:
df_liga['Match Time(CET)'] = pd.to_datetime(df_liga['Match Time(CET)'], format='%Y-%m-%d')
df_liga['Date'] = pd.to_datetime(df_liga['Match Time(CET)'].dt.date, format='%Y-%m-%d') # create a date column
df_liga['Hour'] = df_liga['Match Time(CET)'].dt.hour + df_liga['Match Time(CET)'].dt.minute / 60 # create hour column
df_liga["Month"] = df_liga['Date'].dt.month
df_liga["Day"] = df_liga['Date'].dt.day
df_liga

In [ ]:
mask = (
        (df_liga['Month'] == 6) |
        (df_liga['Month'] == 7) |
        (df_liga['Month'] == 8) |
        (df_liga['Month'] == 9)
)
df_liga = df_liga.loc[mask]
df_liga

In [ ]:
df_liga.to_csv("datasets/clean_laliga_2023.csv")

#### 3.2.3 Create fake fixtures

In [ ]:
df_liga = create_fake_features(df_liga)

In [ ]:
df_liga.head(20)

In [ ]:
all_years_df = extend_years(df_liga, 2002, 2023)

In [ ]:
all_years_df = all_years_df.drop(columns=['Match Time(CET)']) 

In [ ]:
all_years_df

In [ ]:
all_years_df.to_csv('datasets/created_fixtures_2002_2023.csv')

### 3.3 Merge fixtures and locations

In [ ]:
df_liga = pd.read_csv("datasets/created_fixtures_2002_2023.csv") # contains LaLiga summer fixtures for 2002 up to 2022

In [ ]:
df = pd.read_csv('datasets/laliga_rounded.csv') # contains longitude, latitude, and capacity

In [ ]:
merged_df = df_liga.merge(df, left_on='Home Team', right_on='Team', how='inner')
merged_df.drop(["Unnamed: 0_x", "Unnamed: 0_y", "Team"], axis = 1, inplace= True)
merged_df.head()

In [ ]:
merged_df.to_csv("datasets/fixtures_locations.csv")

### 3.4 fill UTCI historical data 

In [ ]:
yearly_df = pd.read_csv("datasets/fixtures_locations.csv")

In [ ]:
#folder_path = "/Volumes/ASRNewVolume_805/dataset-derived-utci-historical-7d82619a-2530-4700-86d8-0f9fb3fd727b"
#folder_path = "/Volumes/ASRNewVolume_805/dataset-derived-utci-historical-837c29ed-9853-4dc0-93a4-3f6c3b080b93"
#folder_path ="/Volumes/ASRNewVolume_805/dataset-derived-utci-historical-8cf94634-325a-4a00-aab9-0e290b98ffef"
folder_path = "/Volumes/ASRNewVolume_805/dataset-derived-utci-historical-486073aa-be98-4d08-bd86-276a57ede642"
nc_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.nc') ]
datasets = {get_date(os.path.basename(f)): xr.open_dataset(f, engine='netcdf4') for f in nc_files} # creates dict, key = date of the dataset, value = content of the dataset



In [ ]:
set_date = set()

for index, row in yearly_df.iterrows():
    date = pd.to_datetime(row['Date'])
    lat = row['latitude']
    lon = row['longitude']
    if date in datasets: 
        value = get_timeseries_at_lat_lon_time(datasets[date], lat, lon, row['valid_time'])
        yearly_df.at[index, 'utci'] = value
    else: # check missing dates 
        set_date.add(date)


In [ ]:
yearly_df = yearly_df.sort_values(by=['Date', 'Hour', 'Home Team'])
yearly_df = yearly_df.reset_index(drop=True)
yearly_df

In [ ]:
yearly_df.to_csv('datasets/UTCI.csv', index=False)

### 3.5 Historical and rcp45 Projections 

In [ ]:
result_df = pd.read_csv("datasets/fixtures_locations.csv", index_col = 0)

In [ ]:
result_df['Year'] = pd.to_datetime(result_df['Date']).dt.year
result_df['timestamp'] = pd.to_datetime(result_df[['Year', 'Month', 'Day', 'Hour']])

In [ ]:
result_df

In [ ]:
root_folders = ['/Volumes/ASRNewVolume_805/historical', '/Volumes/ASRNewVolume_805/rcp45']
for folder_path in root_folders:
    folders = [f for f in os.listdir(folder_path) if f != ".DS_Store"]
    for f in folders:
        result_df = fill_1_year(folder_path +"/"+ f, result_df)

In [ ]:
result_df

In [ ]:
result_df.to_csv("datasets/full_temp_data_2002_2023.csv")

### 3.6 Rcp 4.5 projections 2024-2043

#### 3.6.1 Load Datasets 

In [ ]:
df = pd.read_csv('datasets/laliga_rounded.csv')

In [ ]:
df2 = pd.read_csv("datasets/clean_laliga_2023.csv")

#### 3.6.2 Create fake fixtures

In [ ]:
df2['Match Time(CET)'] = pd.to_datetime(df2['Match Time(CET)'])

In [ ]:
df2_extended = create_fake_features(df2)

In [ ]:
df2_extended["Date"] = pd.to_datetime(df2_extended["Date"], format = "%Y-%m-%d")

In [ ]:
all_years_df = extend_years(df2_extended, 2024, 2044)

In [ ]:
cleaned_df = all_years_df
cleaned_df = cleaned_df.drop(columns=['Match Time(CET)'])
cleaned_df

In [ ]:
cleaned_df.to_csv('datasets/created_fixtures_2024_2044.csv')

#### 3.6.3 Merge fixtures and locations

In [ ]:
df_liga = pd.read_csv("datasets/created_fixtures_2024_2044.csv")

In [ ]:
df = pd.read_csv('datasets/laliga_rounded.csv')

In [ ]:
merged_df = df_liga.merge(df, left_on='Home Team', right_on='Team', how='inner')
merged_df.drop(["Unnamed: 0_y", "Unnamed: 0_x", "Team"], axis = 1, inplace = True)
result_df['Year'] = pd.to_datetime(result_df['Date']).dt.year
result_df['timestamp'] = pd.to_datetime(result_df[['Year', 'Month', 'Day', 'Hour']])
print(merged_df.head())

In [ ]:
merged_df.to_csv("datasets/2024_2044fixtures_locations.csv")

#### 3.6.4 Historical and rcp45 Projections 

In [ ]:
result_df = merged_df

In [ ]:
result_df

In [ ]:
root_folders = [ '/Volumes/ASRNewVolume_805/rcp45_projections']
for folder_path in root_folders:
    folders = [f for f in os.listdir(folder_path) if f != ".DS_Store"]

    for f in folders:
        result_df = fill_1_year(folder_path +"/"+ f, result_df)

In [ ]:
result_df

In [ ]:
result_df.to_csv("datasets/full_temp_data_2024_2044_rcp45.csv")

### 3.7 Rcp 2.6 projections 2024-2043

#### 3.7.1 Historical and rcp26 Projections 

In [ ]:
merged_df = pd.read_csv("datasets/2024_2044fixtures_locations.csv")

In [ ]:
result_df = merged_df

In [ ]:
root_folders = [ '/Volumes/ASRNewVolume_805/rcp26_projections']
for folder_path in root_folders:
    folders = [f for f in os.listdir(folder_path) if f != ".DS_Store"]
    for f in folders:
        result_df = fill_1_year(folder_path +"/"+ f, result_df)

In [ ]:
result_df

In [ ]:
result_df.to_csv("datasets/full_temp_data_2024_2044_rcp26.csv")

### 3.8 Rcp 8.5 projections 2024-2044

In [ ]:
merged_df = pd.read_csv("datasets/2024_2044fixtures_locations.csv")

#### 3.8.1 Historical and rcp26 Projections 

In [ ]:
result_df = merged_df

In [ ]:
root_folders = [ '/Volumes/ASRNewVolume_805/rcp85_projections']
for folder_path in root_folders:
    folders = [f for f in os.listdir(folder_path) if f != ".DS_Store"]
    for f in folders:
        result_df = fill_1_year(folder_path +"/"+ f, result_df)

In [ ]:
result_df

In [ ]:
result_df.to_csv("datasets/full_temp_data_2024_2044_rcp85.csv")

 ## 4. Calcul UTCI

### 4.1 UTCI 2002-2023

In [ ]:
df_past_proj = pd.read_csv("datasets/full_temp_data_2002_2023.csv")

In [ ]:
df_past_proj

In [ ]:
df_past_proj = calculate_UTCI(df_past_proj)

In [ ]:
df_past_proj

In [ ]:
df_past_proj.to_csv("datasets/UTCI_2002_2023_proj_and_histo.csv")

### 4.2 UTCI 2024-2044 4.5

In [ ]:
df_45 = pd.read_csv("datasets/full_temp_data_2024_2044_rcp45.csv")

In [ ]:
df_45

In [ ]:
df_45  = calculate_UTCI(df_45)

In [ ]:
df_45

In [ ]:
df_45.to_csv("datasets/UTCI_2024_2044_45.csv")

### 4.3 UTCI 2024-2044 2.6

In [ ]:
df_26 = pd.read_csv("datasets/full_temp_data_2024_2044_rcp26.csv")

In [ ]:
df_26  = calculate_UTCI(df_26)
df_26

In [ ]:
df_26.to_csv("datasets/UTCI_2024_2044_26.csv")

### 4.4 UTCI 2024-2044 8.5

In [ ]:
df_85 = pd.read_csv("datasets/full_temp_data_2024_2044_rcp85.csv")

In [ ]:
df_85 = calculate_UTCI(df_85)
df_85

In [ ]:
df_85.to_csv("datasets/UTCI_2024_2044_85.csv")

## 5. Descriptive Analysis: What happened ?

In [ ]:
df = pd.read_csv("datasets/UTCI.csv")

In [ ]:
df['utci'] = df['utci'] - 273.15

In [ ]:
df = df.dropna(subset=['utci']) # few missing values, got removed
df = df.sort_values("True_Game", ascending= False).drop_duplicates(["Home Team", 'timestamp'])

In [ ]:
df = categorize_stress(df)

In [ ]:
df[["Home Team", "timestamp","True_Game", "Capacity", 'utci', 'Stress Category']].tail()

In [ ]:
df["policies"] = df["utci"].apply(assign_authorities_conditions)

In [ ]:
location_utci_stats = optimize_clusters(df)

In [ ]:
location_utci_stats = location_utci_stats[["Home Team", "Optimal_Cluster"]]
df = pd.merge(df, location_utci_stats, on='Home Team', how='inner')
df = df.sort_values("True_Game", ascending= False).drop_duplicates(["Home Team", 'timestamp'])
df = df[df["year"] != 2023]

In [ ]:
df.to_csv("datasets/UTCI_cat.csv")

In [ ]:
df = pd.read_csv("datasets/UTCI_cat.csv")

In [ ]:
df

### 5.1 Locations of the fixtures

In [ ]:
df_liga = pd.read_csv("clean_laliga_2023.csv")
plot_locations_histogram(df_liga)

### 5.2 Number of games per hour

In [ ]:
hour_counts_sorted = df_liga['Hour'].value_counts().sort_index()
print(hour_counts_sorted)

In [ ]:
df_liga['Hour'][df_liga['Hour'] == 22] = 21
df_liga['Hour'][df_liga['Hour'] == 20] = 21
df_liga['Hour'][df_liga['Hour'] == 19.5] = 21
df_liga['Hour'][df_liga['Hour'] == 17.5] = 18
df_liga['Hour'][df_liga['Hour'] == 21.5] = 21
df_liga['Hour'][df_liga['Hour'] == 17] = 18
df_liga['Hour'][df_liga['Hour'] == 19] = 18
df_liga['Hour'][df_liga['Hour'] == 16] = 15
df_liga['Hour'][df_liga['Hour'] == 14] = 15
# 21 overlappings
df_liga[["Hour", "month", "day"]].drop_duplicates()

### 5.3 Heat conditions for summer afternoons in Spain

In [ ]:
plot_overall_percentage_stress_category(df,'Percentage of Different Stress Categories for Available Timeslots')

### 5.4 Policies for summer afternoons

In [ ]:
plot_overall_percentage_policies_category(df,'Percentage of Different Policies Categories for Available Timeslots')

### 5.5 Heat Stress conditions for LaLiga games in summer

In [ ]:
# Execute the function with the DataFrame
plot_overall_percentage_stress_category(df[df["True_Game"] == 1],'Percentage of Different Stress Categories for Games Played')

In [ ]:
plot_overall_percentage_policies_category(df[df["True_Game"] == 1],'Percentage of Different Policies Categories for True Games')

### 5.6 Heat stress conditions by locations

In [ ]:
plot_optimal_cluster_percentage_stress_category_histogram(df, 'Percentage Histogram of Stress Categories Grouped by Optimal Cluster')

### 5.7 Heat stress conditions by locations during games

In [ ]:
plot_optimal_cluster_percentage_stress_category_histogram(df[df["True_Game"] == 1], 'Percentage Histogram of Stress Categories during Games Grouped by Optimal Cluster')

### 5.8 Heat Stress conditions by hour 

In [ ]:
plot_grouped_percentage_stress_category_histogram(df)

### 5.9 Heat stress conditions by month

In [ ]:
plot_grouped_month_percentage_stress_category_histogram(df)

### 5.10 Exploratory analysis 

In [ ]:
df[["Date", "True_Game", "Capacity", "utci"]].describe()

In [ ]:
# Plotting UTCI vs Year with different lines for each group

plt.figure(figsize=(15, 8))
sns.lineplot(x='year', y='utci', hue='Optimal_Cluster', data=df, marker="o")

plt.title('UTCI vs Year by Groups')
plt.xlabel('Year')
plt.ylabel('UTCI')
plt.legend(title='Groups', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()


#### 5.10.1 UTCI distribution

In [ ]:
# Creating a histogram of the 'utci' column
plt.figure(figsize=(10, 6))
sns.histplot(df['utci'], kde=False, color='blue')
plt.title('Histogram of UTCI Values')
plt.xlabel('UTCi')
plt.ylabel('Frequency')
plt.grid(False)
plt.gca().set_facecolor('white')

# Saving the histogram to a file
histogram_file_path = 'plots/past_utci_histogram.png'
plt.savefig(histogram_file_path)

####  5.10.2 Boxplots by locations , optimal cluster highlighted

In [ ]:
df.columns

In [ ]:
df = df.sort_values(['Optimal_Cluster', 'Home Team'])

# Setting up the plot
plt.figure(figsize=(10, 6))
sns.boxplot(x='Home Team', y='utci', hue='Optimal_Cluster', data=df, palette='Set2')
plt.xticks(rotation=90)
plt.title('UTCI Values Colored by Optimal Cluster')
plt.tight_layout()
plt.grid(False)
plt.gca().set_facecolor('white')

# Saving the plot
file_path = 'plots/past_utci_boxplot_by_optimal_cluster.png'
plt.savefig(file_path)

file_path


## 6. Prescriptive Analysis: What will happen ? 

### 6.1 Data merging and cleaning

In [ ]:
df_85 = pd.read_csv("datasets/UTCI_2024_2044_85.csv")
df_45 = pd.read_csv('datasets/UTCI_2024_2044_45.csv')
df_26 = pd.read_csv('datasets/UTCI_2024_2044_26.csv')

In [ ]:
df_85.columns

In [ ]:
df_85 = df_85[["Home Team", "timestamp", "True_Game", "UTCI"]]
df_85.rename(columns={'UTCI': 'UTCI_85'}, inplace=True)
df_45 = df_45[["Home Team", "timestamp", "True_Game", "UTCI"]]
df_45.rename(columns={'UTCI': 'UTCI_45'}, inplace=True)
df_26.rename(columns={'UTCI': 'UTCI_26'}, inplace=True)

In [ ]:
df_26

In [ ]:
merging_df = pd.merge(df_26, df_45, how = 'inner', on = ['Home Team', "timestamp"])

In [ ]:
merging_df = pd.merge(merging_df, df_85, how = 'inner', on = ['Home Team', "timestamp"])
merging_df

### 6.2 Download of the bias corrected version

In [ ]:
future_df = pd.read_csv("datasets/corrected_future_UTCI.csv")
future_df = future_df.rename(columns={
    'utci_bc_rcp26': 'UTCI_26',
    'utci_bc_rcp45': 'UTCI_45',
    'utci_bc_rcp85': 'UTCI_85'
})
# Convert 'timestamp' to datetime for consistent comparison
future_df["timestamp"] = pd.to_datetime(future_df['timestamp'])
future_df["hour"] = pd.to_datetime(future_df['timestamp']).dt.hour
future_df["month"] = pd.to_datetime(future_df['timestamp']).dt.month
future_df["day"] = pd.to_datetime(future_df['timestamp']).dt.day
future_df["year"] = pd.to_datetime(future_df['timestamp']).dt.year

In [ ]:
# corrects issues whith True_Game
rows_to_remove = pd.DataFrame()

for year in future_df["year"].unique():
    # Update the "Hour" field for specific conditions
    future_df.loc[(future_df["Home Team"] == "Betis Seville") &
                  (future_df["timestamp"] == pd.Timestamp(f"{year}-08-15 21:30:00")), "hour"] = 22

    future_df.loc[(future_df["Home Team"] == "Valence FC") &
                  (future_df["timestamp"] == pd.Timestamp(f"{year}-08-14 19:30:00")), "hour"] = 20

    future_df.loc[(future_df["Home Team"] == "Getafe") &
                  (future_df["timestamp"] == pd.Timestamp(f"{year}-08-15 19:30:00")), "hour"] = 20

    future_df.loc[(future_df["Home Team"] == "Majorque") &
                  (future_df["timestamp"] == pd.Timestamp(f"{year}-08-20 19:30:00")), "hour"] = 20

    future_df.loc[(future_df["Home Team"] == "Atlético de Madrid") &
                  (future_df["timestamp"] == pd.Timestamp(f"{year}-08-21 19:30:00")), "hour"] = 20

    future_df.loc[(future_df["Home Team"] == "Rayo Vallecano") &
                  (future_df["timestamp"] == pd.Timestamp(f"{year}-08-27 19:30:00")), "hour"] = 20

    future_df.loc[(future_df["Home Team"] == "FC Barcelone") &
                  (future_df["timestamp"] == pd.Timestamp(f"{year}-08-28 19:30:00")), "hour"] = 20

    # For "Cadix"
    condition1 = future_df['Home Team'] == "Cadix"
    condition2 = future_df['timestamp'] == pd.Timestamp(f"{year}-08-14 18:00:00")
    rows_to_remove = rows_to_remove.append(future_df[condition1 & condition2])
    future_df.loc[(condition1) & (pd.to_datetime(future_df["timestamp"]) == pd.to_datetime(f"{year}-08-14 17:30:00")), "hour"] = 18

    # For "Athletic Bilbao"
    condition1 = future_df['Home Team'] == "Athletic Bilbao"
    condition2 = future_df['timestamp'] == pd.Timestamp(f"{year}-08-15 18:00:00")
    rows_to_remove = rows_to_remove.append(future_df[condition1 & condition2])
    future_df.loc[(condition1) & (pd.to_datetime(future_df["timestamp"]) == pd.to_datetime(f"{year}-08-15 17:30:00")), "hour"] = 18

    condition2 = future_df['timestamp'] == pd.Timestamp(f"{year}-08-20 18:00:00")
    rows_to_remove = rows_to_remove.append(future_df[condition1 & condition2])
    future_df.loc[(condition1) & (pd.to_datetime(future_df["timestamp"]) == pd.to_datetime(f"{year}-08-20 17:30:00")), "hour"] = 18

    condition2 = future_df['timestamp'] == pd.Timestamp(f"{year}-08-21 18:00:00")
    rows_to_remove = rows_to_remove.append(future_df[condition1 & condition2])
    future_df.loc[(condition1) & (pd.to_datetime(future_df["timestamp"]) == pd.to_datetime(f"{year}-08-21 17:30:00")), "hour"] = 18

    # For "Osasuna"
    condition1 = future_df['Home Team'] == "Osasuna"
    rows_to_remove = rows_to_remove.append(future_df[condition1 & condition2])
    future_df.loc[(condition1) & (pd.to_datetime(future_df["timestamp"]) == pd.to_datetime(f"{year}-08-20 17:30:00")), "hour"] = 18
    
    #For "Elche Club de Futbol"
    condition1 = future_df['Home Team'] == "Elche Club de Futbol"
    condition2 = future_df['timestamp'] == pd.Timestamp(f"{year}-08-27 18:00:00")
    rows_to_remove = rows_to_remove.append(future_df[condition1 & condition2])
    future_df.loc[(condition1) & (pd.to_datetime(future_df["timestamp"]) == pd.to_datetime(f"{year}-08-27 17:30:00")), "hour"] = 18

    #For "Getafe"
    condition1 = future_df['Home Team'] == "Getafe"
    condition2 = future_df['timestamp'] == pd.Timestamp(f"{year}-08-28 18:00:00")
    rows_to_remove = rows_to_remove.append(future_df[condition1 & condition2])
    future_df.loc[(condition1) & (pd.to_datetime(future_df["timestamp"]) == pd.to_datetime(f"{year}-08-28 17:30:00")), "hour"] = 18

# Drop the collected rows from `future_df`
if not rows_to_remove.empty:
    future_df = future_df.drop(rows_to_remove.index).reset_index(drop=True)


In [ ]:
future_df = future_df.sort_values(["True_Game"], ascending = False)
future_df = future_df.drop_duplicates(["Home Team", "month", 'day', 'hour', 'year'])


In [ ]:
future_df.sort_values(['Home Team', "month", "day", "hour"])

#### 6.2.1 compare data before and after bias correction

In [ ]:
before_bias_correction = merging_df
after_bias_correction = future_df

In [ ]:
# Set the palette for the different datasets
palette = ["#1f77b4", "#ff7f0e"]  # You can choose different colors

# Set the style to 'white' which removes the grid and has a white background
sns.set_style("white")
# Prepare the data for plotting
data_to_plot = pd.concat([
    before_bias_correction.melt(value_vars=['UTCI_26', 'UTCI_45', 'UTCI_85'], var_name='Variable', value_name='Value').assign(Dataset='before_bias_correction'),
    after_bias_correction.melt(value_vars=['UTCI_26', 'UTCI_45', 'UTCI_85'], var_name='Variable', value_name='Value').assign(Dataset='after_bias_correction')
])


# Plotting
plt.figure(figsize=(10, 6))
sns.boxplot(x='Variable', y='Value', hue='Dataset', data=data_to_plot, palette=palette)
plt.title('Box plot of UTCI_26, UTCI_45, and UTCI_85 from before and after bias correction')
plt.xlabel('Variable')
plt.ylabel('Value')
plt.legend(title='Dataset')
plt.savefig("plots/bias_correction.png")
plt.show()


In [ ]:
future_df = future_df[["Home Team", "True_Game", "Stadium", "latitude", "longitude", "Capacity", "Division", "timestamp", "UTCI_26", "UTCI_45", "UTCI_85", 'hour', 'month','day', 'year']]
future_df = categorize_stress_future(future_df, "UTCI_26", "Stress Category 26")
future_df = categorize_stress_future(future_df, "UTCI_45", "Stress Category 45")
future_df = categorize_stress_future(future_df, "UTCI_85", "Stress Category 85")
#merging_df.to_csv("full_UTCI_comparison_2024_2044.csv")

In [ ]:
na_count = future_df['UTCI_26'].isna().sum()
print(f"Number of NA values in the column: {na_count}")

In [ ]:
future_df = future_df.dropna(subset=['UTCI_26']) # remove pcq sinon t'as des NA dans les percenatges

In [ ]:
len(future_df)

In [ ]:
for scenario in ["26", "45", "85"]:
    future_df[f'policies_rcp{scenario}'] = future_df[f"UTCI_{scenario}"].apply(assign_authorities_conditions)

In [ ]:
future_df = pd.merge(future_df, location_utci_stats, on='Home Team', how='inner')

In [ ]:
future_df

In [ ]:
future_df.to_csv("datasets/full_UTCI_comparison_2024_2044_corrected.csv")


In [ ]:
future_df = pd.read_csv("datasets/full_UTCI_comparison_2024_2044_corrected.csv")

In [ ]:
future_df[["True_Game", "Capacity", "UTCI_26", "UTCI_45", "UTCI_85"]].describe()

### 6.3 Comparison

In [ ]:
# List of columns to plot
stress_columns = ['Stress Category 26', 'Stress Category 45', 'Stress Category 85']

# Call the function with the DataFrame and the list of stress category columns
plot_stress_category_distribution_colored(future_df, stress_columns)

In [ ]:
future_df.columns

In [ ]:
# Example usage of the function
plot_stress_category_distribution_with_values(future_df,  'Stress Category 26', 'UTCI 2.6 Stress Category Distribution by Month')
plot_stress_category_distribution_with_values(future_df, 'Stress Category 45', 'UTCI 4.5 Stress Category Distribution by Month')
plot_stress_category_distribution_with_values(future_df,  'Stress Category 85', 'UTCI 8.5 Stress Category Distribution by Month')

In [ ]:
# Example usage of the function for UTCI categories by hour
plot_stress_category_distribution_by_hour(future_df, 'UTCI_26', 'Stress Category 26', 'UTCI 2.6 Stress Category Distribution by Hour')
plot_stress_category_distribution_by_hour(future_df, 'UTCI_45', 'Stress Category 45', 'UTCI 4.5 Stress Category Distribution by Hour')
plot_stress_category_distribution_by_hour(future_df, 'UTCI_85', 'Stress Category 85', 'UTCI 8.5 Stress Category Distribution by Hour')

In [ ]:
order = ['No thermal stress', 'Moderate heat stress', 'Strong heat stress',
         'Very strong heat stress', 'Extreme heat stress', 'Slight cold stress']
# Generating and saving the plots for each RCP with colors
plot_and_save_stress_category_percentage_ordered(future_df, 'Stress Category 26', '2.6', "plots/rcp26_clusters.pdf", color_mapping, order)
plot_and_save_stress_category_percentage_ordered(future_df, 'Stress Category 45', '4.5', "plots/rcp45_clusters.pdf", color_mapping, order)
plot_and_save_stress_category_percentage_ordered(future_df, 'Stress Category 85', '8.5', "plots/rcp85_clusters.pdf", color_mapping, order)

## 7. Optimization Problem

### 7.1 RCP 8.5

In [ ]:
df = pd.read_csv("datasets/full_UTCI_comparison_2024_2044_corrected.csv")

In [ ]:
df

In [ ]:
df["Home Team"] = df["Home Team"].apply(remove_accents)

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date
df['hour'] = df['timestamp'].dt.hour
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
temp = df[["Home Team",  "hour", "day", "month"]].drop_duplicates()

In [ ]:
df[df["True_Game"] == 1][["Home Team", "month", "day", "hour", 'True_Game']].drop_duplicates()

In [ ]:
df['hour'][df['hour'] == 22] = 21
df['hour'][df['hour'] == 20] = 21
df['hour'][df['hour'] == 17] = 18
df['hour'][df['hour'] == 19] = 18
df['hour'][df['hour'] == 16] = 15
df['hour'][df['hour'] == 14] = 15

df_temp = df[df["True_Game"] == 1][["Home Team", "month", "day", "hour", 'True_Game']].drop_duplicates()
nb_by_hour = pd.DataFrame()
nb_by_hour["hour"] = df_temp["hour"].drop_duplicates()
nb_by_hour['nb_of_occurences']= df_temp['hour'].value_counts().values
nb_by_hour


In [ ]:
df_copy = df

In [ ]:
# Convert 'timestamp' to datetime and extract date and hour
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date
df['hour'] = df['timestamp'].dt.hour
df['month'] = df['timestamp'].dt.month

# Create a mapping of stress categories to numerical values
stress_value_map = {
    'No thermal stress': 0,
    'Moderate heat stress': 1,
    'Strong heat stress': 2,
    'Very strong heat stress': 3,
    'Extreme heat stress': 4
}

# Map the stress categories in 'Stress Category 85' to their numerical values
df['Stress Value 85'] = df['Stress Category 85'].map(stress_value_map)

# Group by location (latitude, longitude), date, and hour, and calculate the sum of average cumulated percentages
r_df = df.groupby(['Home Team', 'month','hour'])['Stress Value 85'].mean().reset_index()

# Rename the columns for clarity
r_df.rename(columns={'Stress Value 85': 'Average Stress Value 85'}, inplace=True)

r_df


In [ ]:
# Convert 'date' column to datetime format to extract day and month
unique_combinations_df = df_copy
unique_combinations_df['date'] = pd.to_datetime(unique_combinations_df['timestamp'])
unique_combinations_df['day'] = unique_combinations_df['date'].dt.day
unique_combinations_df['month'] = unique_combinations_df['date'].dt.month

# Drop the 'date' column as we have separated it into 'day' and 'month'
unique_combinations_df.drop('date', axis=1, inplace=True)

# Remove duplicate rows
unique_combinations_df.drop_duplicates(inplace=True)

x_df = unique_combinations_df[["Home Team", "day", "month", "hour"]]
x_df.drop_duplicates(inplace=True)
x_df

In [ ]:
o_df = x_df

In [ ]:
df_short = df[["Home Team", "Capacity"]].drop_duplicates()

In [ ]:
df_short

In [ ]:
# Create a dictionary with keys as home team and values as the capacity
capacities_dict = df_short.set_index('Home Team')['Capacity'].to_dict()

In [ ]:
# Create a new model
model = gp.Model("linear_problem")

In [ ]:

o_df['key'] = list(zip(o_df["Home Team"], o_df["month"], o_df["day"], o_df["hour"]))
x_df["key"] = list(zip(x_df["Home Team"], x_df["month"], x_df["day"], x_df["hour"]))

# Extract the unique keys
unique_keys_o = o_df['key'].unique()
unique_keys_x = x_df["key"].unique()


# Create binary variables o_d_l using unique keys
x_h_d_l = model.addVars(unique_keys_x, vtype=GRB.BINARY, name="x")
o_d_l = model.addVars(unique_keys_o, vtype=GRB.BINARY, name="o")



value_column_name = r_df.columns[-1]
# Create a dictionary for the values
values_dict = {(row['hour'], row['month'], row['Home Team']): row[value_column_name]
               for index, row in r_df.iterrows()}
model.update()

In [ ]:
# Initialize an empty dictionary to hold the mapping
team_month_day_hours = {}

# Iterate through the DataFrame to populate the dictionary
for index, row in df.iterrows():
    home_team = row['Home Team']
    day = row["day"]
    month = row["month"]
    hour = row['hour']

    # Initialize the nested dictionaries if the 'Home Team' or month or day is not already in the dictionary
    if home_team not in team_month_day_hours:
        team_month_day_hours[home_team] = {}
    if month not in team_month_day_hours[home_team]:
        team_month_day_hours[home_team][month] = {}
    if day not in team_month_day_hours[home_team][month]:
        team_month_day_hours[home_team][month][day] = []

    # Append the hour to the list of hours available for that 'Home Team', month, and day
    if hour not in team_month_day_hours[home_team][month][day]:
        team_month_day_hours[home_team][month][day].append(hour)



In [ ]:

unique_keys_x_list = unique_keys_x.tolist()
for home_team, months in team_month_day_hours.items():
    for month, days in months.items():
        for day, hours_list in days.items():
            # Ensure that home_team, month, and day are in the correct type before comparison
            home_team_str = str(home_team)  
            month_int = int(month)
            day_int = int(day)

            vars_in_constraint = [x_h_d_l[home_team_str, month_int, day_int, h] for h in hours_list if (home_team_str, month_int, day_int, h) in unique_keys_x_list]
            model.addConstr(sum(vars_in_constraint) == 1, "daily_hour_sum_constraint_{}_{}_{}".format(home_team, month, day))




for hour in nb_by_hour["hour"]:
    print(hour, nb_by_hour[nb_by_hour['hour'] == hour]['nb_of_occurences'].values[0])
    sum_x_vars = sum(x_h_d_l[l, m, d, hour] for l, m, d, h in x_h_d_l.keys() if h == hour)
    model.addConstr(sum_x_vars == nb_by_hour[nb_by_hour['hour'] == hour]['nb_of_occurences'].values[0], f"hour_constraint_{hour}")
    
# Define a large M value
M = 1000

# Iterate over each unique combination of location, month, day, and hour
for l, month_days in team_month_day_hours.items():
    for m, days in month_days.items():
        for d, hours_list in days.items():
            for h in hours_list:
                sum_x_vars = sum(x_h_d_l[l2, m, d, h] for l2 in team_month_day_hours if (l2, m, d, h) in unique_keys_x_list)
            
                # Constraint 1: If two or more games are scheduled, overlapping must be 1
                model.addConstr(sum_x_vars <= 1 + M * o_d_l[l, m, d, h],
                                f"overlap_max_constraint_{l}_{m}_{d}_{h}")
                
                # Constraint 2: If less than two games are scheduled, overlapping must be 0
                model.addConstr(sum_x_vars >= 2 - M * (1 - o_d_l[l, m, d, h]),
                                f"overlap_min_constraint_{l}_{m}_{d}_{h}")
                
model.update()



In [ ]:
# Initialize the objective function as a linear expression because r_h_m_l are constants
objective = gp.LinExpr()

i = 0
# Add the sum of products of x_h_d_l and r_h_m_l for matching hour, location, and month
for x_key in x_h_d_l.keys():
    
    
    l, m, d, h = x_key  
    r_key = (h, m, l)
    if r_key in values_dict:  # values_dict contains fixed values for r_h_m_l
        i += 1
        objective.add(x_h_d_l[x_key] * values_dict[r_key] * 0.99995)
       
# Add the sum of products of o_d_l and c_l for matching location
for o_key in o_d_l.keys():
    
    l,d,m,h = o_key  # o_key is a tuple (d, l)
    if l in capacities_dict:  # capacities_dict contains the capacities corresponding to c_l
        i+=1
        objective.add(o_d_l[o_key] * capacities_dict[l] * 0.00005)


# Set the objective function in the model
model.update()
model.setObjective(objective, GRB.MINIMIZE)

In [ ]:
num_terms = objective.size()
print("Number of terms in the objective function:", num_terms)

In [ ]:
## Optimize the model
model.optimize()

In [ ]:
# List to hold the data
data = []

# Check if the solution exists
if model.status == GRB.OPTIMAL:
    # Iterate over the variables
    for v in model.getVars():
        # Check if the variable value is 1
        if round(v.x) == 1:
            # Parse the variable name
            print(v.varName.split(','))
            team, month, day, hour = v.varName.split(',')
            # Add to the list
            if team[0] == 'x':
                team = team[2:]
                hour = hour[:-1]
                data.append([team, int(month), int(day), int(hour)])

    # Create a DataFrame
    df_res = pd.DataFrame(data, columns=['Team', 'Month', 'Day', 'Hour'])
else:
    print("No optimal solution found.")

In [ ]:
df_res

In [ ]:
df_res["optimal_schedule"] = 1

In [ ]:
df_res

In [ ]:
merge_df = pd.merge(df, df_res, left_on=["Home Team", "hour", "month", "day"], right_on=["Team", "Hour", "Month", "Day"], how="inner")

In [ ]:
merge_df

In [ ]:
plot_overall_percentage_stress_category(merge_df,'Percentage of Different Stress Categories for optimal game features', 'Stress Category 85')

In [ ]:
plot_overall_percentage_stress_category(df[df["True_Game"] == 1],'Percentage of Different Stress Categories for non optimal game features', 'Stress Category 85')

In [ ]:
plot_overall_percentage_policies_category(merge_df,'Percentage of Different Policies Categories for Available Timeslots', "policies_rcp85")

In [ ]:
plot_overall_percentage_policies_category(df[df["True_Game"] == 1],'Percentage of Different Policies Categories for Available Timeslots', "policies_rcp85")

In [ ]:
plot_overall_percentage_policies_category(df,'Percentage of Different Policies Categories for Available Timeslots', "policies_rcp85")

### 7.2 RCP 4.5 

In [ ]:
df = pd.read_csv("datasets/full_UTCI_comparison_2024_2044_corrected.csv")

In [ ]:
df

In [ ]:
df["Home Team"] = df["Home Team"].apply(remove_accents)

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date
df['hour'] = df['timestamp'].dt.hour
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
temp = df[["Home Team",  "hour", "day", "month"]].drop_duplicates()

In [ ]:
df[df["True_Game"] == 1][["Home Team", "month", "day", "hour", 'True_Game']].drop_duplicates()

In [ ]:
df['hour'][df['hour'] == 22] = 21
df['hour'][df['hour'] == 20] = 21
df['hour'][df['hour'] == 17] = 18
df['hour'][df['hour'] == 19] = 18
df['hour'][df['hour'] == 16] = 15
df['hour'][df['hour'] == 14] = 15

df_temp = df[df["True_Game"] == 1][["Home Team", "month", "day", "hour", 'True_Game']].drop_duplicates()
nb_by_hour = pd.DataFrame()
nb_by_hour["hour"] = df_temp["hour"].drop_duplicates()
nb_by_hour['nb_of_occurences']= df_temp['hour'].value_counts().values
nb_by_hour


In [ ]:
df_copy = df

In [ ]:
# Convert 'timestamp' to datetime and extract date and hour
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date
df['hour'] = df['timestamp'].dt.hour
df['month'] = df['timestamp'].dt.month

# Create a mapping of stress categories to numerical values
stress_value_map = {
    'No thermal stress': 0,
    'Moderate heat stress': 1,
    'Strong heat stress': 2,
    'Very strong heat stress': 3,
    'Extreme heat stress': 4
}

# Map the stress categories in 'Stress Category 4.5' to their numerical values
df['Stress Value 45'] = df['Stress Category 45'].map(stress_value_map)
# Group by location (latitude, longitude), date, and hour, and calculate the sum of average cumulated percentages
r_df = df.groupby(['Home Team', 'month','hour'])['Stress Value 45'].mean().reset_index()

# Rename the columns for clarity
r_df.rename(columns={'Stress Value 45': 'Average Stress Value 45'}, inplace=True)

r_df


In [ ]:
# Convert 'date' column to datetime format to extract day and month
unique_combinations_df = df_copy
unique_combinations_df['date'] = pd.to_datetime(unique_combinations_df['timestamp'])
unique_combinations_df['day'] = unique_combinations_df['date'].dt.day
unique_combinations_df['month'] = unique_combinations_df['date'].dt.month

# Drop the 'date' column as we have separated it into 'day' and 'month'
unique_combinations_df.drop('date', axis=1, inplace=True)

# Remove duplicate rows
unique_combinations_df.drop_duplicates(inplace=True)

x_df = unique_combinations_df[["Home Team", "day", "month", "hour"]]
x_df.drop_duplicates(inplace=True)
x_df

In [ ]:
o_df = x_df

In [ ]:
df_short = df[["Home Team", "Capacity"]].drop_duplicates()

In [ ]:
df_short

In [ ]:
# Create a dictionary with keys as home team and values as the capacity
capacities_dict = df_short.set_index('Home Team')['Capacity'].to_dict()

In [ ]:
# Create a new model
model = gp.Model("linear_problem")

In [ ]:

o_df['key'] = list(zip(o_df["Home Team"], o_df["month"], o_df["day"], o_df["hour"]))
x_df["key"] = list(zip(x_df["Home Team"], x_df["month"], x_df["day"], x_df["hour"]))

# Extract the unique keys
unique_keys_o = o_df['key'].unique()
unique_keys_x = x_df["key"].unique()


# Create binary variables o_d_l using unique keys
x_h_d_l = model.addVars(unique_keys_x, vtype=GRB.BINARY, name="x")
o_d_l = model.addVars(unique_keys_o, vtype=GRB.BINARY, name="o")

value_column_name = r_df.columns[-1]
# Create a dictionary for the values
values_dict = {(row['hour'], row['month'], row['Home Team']): row[value_column_name]
               for index, row in r_df.iterrows()}

print("vdict",values_dict)
model.update()

In [ ]:
# Initialize an empty dictionary to hold the mapping
team_month_day_hours = {}

# Iterate through the DataFrame to populate the dictionary
for index, row in df.iterrows():
    home_team = row['Home Team']
    day = row["day"]
    month = row["month"]
    hour = row['hour']

    # Initialize the nested dictionaries if the 'Home Team' or month or day is not already in the dictionary
    if home_team not in team_month_day_hours:
        team_month_day_hours[home_team] = {}
    if month not in team_month_day_hours[home_team]:
        team_month_day_hours[home_team][month] = {}
    if day not in team_month_day_hours[home_team][month]:
        team_month_day_hours[home_team][month][day] = []

    # Append the hour to the list of hours available for that 'Home Team', month, and day
    if hour not in team_month_day_hours[home_team][month][day]:
        team_month_day_hours[home_team][month][day].append(hour)

print('full dict', team_month_day_hours)



In [ ]:
# Now we can use this mapping to add constraints to the model

# Now we can use this mapping to add constraints to the model
unique_keys_x_list = unique_keys_x.tolist()
for home_team, months in team_month_day_hours.items():
    for month, days in months.items():
        for day, hours_list in days.items():
            # Ensure that home_team, month, and day are in the correct type before comparison
            home_team_str = str(home_team)  # or int(home_team) based on your data
            month_int = int(month)
            day_int = int(day)

            vars_in_constraint = [x_h_d_l[home_team_str, month_int, day_int, h] for h in hours_list if (home_team_str, month_int, day_int, h) in unique_keys_x_list]
            #print(f"Constraint for {home_team} on {month}/{day} includes variables: {vars_in_constraint}")
            model.addConstr(sum(vars_in_constraint) == 1, "daily_hour_sum_constraint_{}_{}_{}".format(home_team, month, day))




for hour in nb_by_hour["hour"]:
    print(hour, nb_by_hour[nb_by_hour['hour'] == hour]['nb_of_occurences'].values[0])
    sum_x_vars = sum(x_h_d_l[l, m, d, hour] for l, m, d, h in x_h_d_l.keys() if h == hour)
    model.addConstr(sum_x_vars == nb_by_hour[nb_by_hour['hour'] == hour]['nb_of_occurences'].values[0], f"hour_constraint_{hour}")
    
# Define a large M value
M = 1000

# Iterate over each unique combination of location, month, day, and hour
for l, month_days in team_month_day_hours.items():
    for m, days in month_days.items():
        for d, hours_list in days.items():
            for h in hours_list:
                sum_x_vars = sum(x_h_d_l[l2, m, d, h] for l2 in team_month_day_hours if (l2, m, d, h) in unique_keys_x_list)
            
                # Constraint 1: If two or more games are scheduled, overlapping must be 1
                model.addConstr(sum_x_vars <= 1 + M * o_d_l[l, m, d, h],
                                f"overlap_max_constraint_{l}_{m}_{d}_{h}")
                
                # Constraint 2: If less than two games are scheduled, overlapping must be 0
                model.addConstr(sum_x_vars >= 2 - M * (1 - o_d_l[l, m, d, h]),
                                f"overlap_min_constraint_{l}_{m}_{d}_{h}")
                
model.update()



In [ ]:
# Initialize the objective function as a linear expression because r_h_m_l are constants
objective = gp.LinExpr()

i = 0
# Add the sum of products of x_h_d_l and r_h_m_l for matching hour, location, and month
for x_key in x_h_d_l.keys():
    
    
    l, m, d, h = x_key 
    r_key = (h, m, l)

    if r_key in values_dict:  # values_dict contains fixed values for r_h_m_l
        
        i += 1
        objective.add(x_h_d_l[x_key] * values_dict[r_key] * 0.99995)    
# Add the sum of products of o_d_l and c_l for matching location
for o_key in o_d_l.keys():
    
    l,d,m,h = o_key  # o_key is a tuple (d, l)
    if l in capacities_dict:  # capacities_dict contains the capacities corresponding to c_l
        i+=1
        objective.add(o_d_l[o_key] * capacities_dict[l] * 0.00005)


# Set the objective function in the model
model.update()
model.setObjective(objective, GRB.MINIMIZE)

In [ ]:
num_terms = objective.size()
print("Number of terms in the objective function:", num_terms)



In [ ]:
# Print all constraints safely, handling potential encoding errors
print("\nConstraints:")
constraints = model.getConstrs()
for constr in constraints:
    lhs_expression = model.getRow(constr)
    rhs_value = constr.RHS
    print(f"{constr.ConstrName}: {lhs_expression} = {rhs_value}")



In [ ]:
## Optimize the model
model.optimize()

In [ ]:
if model.status == GRB.OPTIMAL:
    # Retrieve the values of the variables
    for v in model.getVars():
        print(f'{v.varName}: {v.x}')

In [ ]:
# List to hold the data
data = []

# Check if the solution exists
if model.status == GRB.OPTIMAL:
    # Iterate over the variables
    for v in model.getVars():
        # Check if the variable value is 1
        if round(v.x) == 1:
            # Parse the variable name
            print(v.varName.split(','))
            team, month, day, hour = v.varName.split(',')
            # Add to the list
            if team[0] == 'x':
                team = team[2:]
                hour = hour[:-1]
                data.append([team, int(month), int(day), int(hour)])
        

    # Create a DataFrame
    df_res = pd.DataFrame(data, columns=['Team', 'Month', 'Day', 'Hour'])
else:
    print("No optimal solution found.")

In [ ]:
df_res

In [ ]:
df_res["optimal_schedule"] = 1

In [ ]:
df_res

In [ ]:
df

In [ ]:
merge_df = pd.merge(df, df_res, left_on=["Home Team", "hour", "month", "day"], right_on=["Team", "Hour", "Month", "Day"], how="inner")

In [ ]:
merge_df

In [ ]:
merge_df["Stress Category 45"].unique()

In [ ]:
plot_overall_percentage_stress_category(merge_df,'Percentage of Different Stress Categories for optimal game features', 'Stress Category 45')

In [ ]:
plot_overall_percentage_stress_category(df[df["True_Game"] == 1],'Percentage of Different Stress Categories for non optimal game features', 'Stress Category 45')

In [ ]:
plot_overall_percentage_policies_category(merge_df,'Percentage of Different Policies Categories for Available Timeslots', "policies_rcp45")

In [ ]:
plot_overall_percentage_policies_category(df[df["True_Game"] == 1],'Percentage of Different Policies Categories for Available Timeslots', "policies_rcp45")

In [ ]:
plot_overall_percentage_policies_category(df,'Percentage of Different Policies Categories for Available Timeslots', "policies_rcp45")

### 7.3 RCP 2.6

In [ ]:
df = pd.read_csv("datasets/full_UTCI_comparison_2024_2044_corrected.csv")

In [ ]:
df

In [ ]:
df["Home Team"] = df["Home Team"].apply(remove_accents)

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date
df['hour'] = df['timestamp'].dt.hour
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
temp = df[["Home Team",  "hour", "day", "month"]].drop_duplicates()

In [ ]:
df[df["True_Game"] == 1][["Home Team", "month", "day", "hour", 'True_Game']].drop_duplicates()

In [ ]:
df['hour'][df['hour'] == 22] = 21
df['hour'][df['hour'] == 20] = 21
df['hour'][df['hour'] == 17] = 18
df['hour'][df['hour'] == 19] = 18
df['hour'][df['hour'] == 16] = 15
df['hour'][df['hour'] == 14] = 15

df_temp = df[df["True_Game"] == 1][["Home Team", "month", "day", "hour", 'True_Game']].drop_duplicates()
nb_by_hour = pd.DataFrame()
nb_by_hour["hour"] = df_temp["hour"].drop_duplicates()
nb_by_hour['nb_of_occurences']= df_temp['hour'].value_counts().values
nb_by_hour


In [ ]:
df_copy = df

In [ ]:
# Convert 'timestamp' to datetime and extract date and hour
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date
df['hour'] = df['timestamp'].dt.hour
df['month'] = df['timestamp'].dt.month

# Create a mapping of stress categories to numerical values
stress_value_map = {
    'No thermal stress': 0,
    'Moderate heat stress': 1,
    'Strong heat stress': 2,
    'Very strong heat stress': 3,
    'Extreme heat stress': 4
}

# Map the stress categories in 'Stress Category 26' to their numerical values
df['Stress Value 26'] = df['Stress Category 26'].map(stress_value_map)

# Group by location (latitude, longitude), date, and hour, and calculate the sum of average cumulated percentages
r_df = df.groupby(['Home Team', 'month','hour'])['Stress Value 26'].mean().reset_index()

# Rename the columns for clarity
r_df.rename(columns={'Stress Value 26': 'Average Stress Value 26'}, inplace=True)

r_df


In [ ]:
# Convert 'date' column to datetime format to extract day and month
unique_combinations_df = df_copy
unique_combinations_df['date'] = pd.to_datetime(unique_combinations_df['timestamp'])
unique_combinations_df['day'] = unique_combinations_df['date'].dt.day
unique_combinations_df['month'] = unique_combinations_df['date'].dt.month

# Drop the 'date' column as we have separated it into 'day' and 'month'
unique_combinations_df.drop('date', axis=1, inplace=True)

# Remove duplicate rows
unique_combinations_df.drop_duplicates(inplace=True)

x_df = unique_combinations_df[["Home Team", "day", "month", "hour"]]
x_df.drop_duplicates(inplace=True)
x_df

In [ ]:
o_df = x_df

In [ ]:
df_short = df[["Home Team", "Capacity"]].drop_duplicates()

In [ ]:
df_short

In [ ]:
# Create a dictionary with keys as home team and values as the capacity
capacities_dict = df_short.set_index('Home Team')['Capacity'].to_dict()



In [ ]:
# Create a new model
model = gp.Model("linear_problem")

In [ ]:

o_df['key'] = list(zip(o_df["Home Team"], o_df["month"], o_df["day"], o_df["hour"]))
x_df["key"] = list(zip(x_df["Home Team"], x_df["month"], x_df["day"], x_df["hour"]))

# Extract the unique keys
unique_keys_o = o_df['key'].unique()
unique_keys_x = x_df["key"].unique()

# Create binary variables o_d_l using unique keys
x_h_d_l = model.addVars(unique_keys_x, vtype=GRB.BINARY, name="x")
o_d_l = model.addVars(unique_keys_o, vtype=GRB.BINARY, name="o")

value_column_name = r_df.columns[-1]
# Create a dictionary for the values
values_dict = {(row['hour'], row['month'], row['Home Team']): row[value_column_name]
               for index, row in r_df.iterrows()}

print("vdict",values_dict)
                
model.update()



In [ ]:
# Initialize an empty dictionary to hold the mapping
team_month_day_hours = {}

# Iterate through the DataFrame to populate the dictionary
for index, row in df.iterrows():
    home_team = row['Home Team']
    day = row["day"]
    month = row["month"]
    hour = row['hour']

    # Initialize the nested dictionaries if the 'Home Team' or month or day is not already in the dictionary
    if home_team not in team_month_day_hours:
        team_month_day_hours[home_team] = {}
    if month not in team_month_day_hours[home_team]:
        team_month_day_hours[home_team][month] = {}
    if day not in team_month_day_hours[home_team][month]:
        team_month_day_hours[home_team][month][day] = []

    # Append the hour to the list of hours available for that 'Home Team', month, and day
    if hour not in team_month_day_hours[home_team][month][day]:
        team_month_day_hours[home_team][month][day].append(hour)

print('full dict', team_month_day_hours)



In [ ]:
# Now we can use this mapping to add constraints to the model

# Now we can use this mapping to add constraints to the model
unique_keys_x_list = unique_keys_x.tolist()
for home_team, months in team_month_day_hours.items():
    for month, days in months.items():
        for day, hours_list in days.items():
            # Ensure that home_team, month, and day are in the correct type before comparison
            home_team_str = str(home_team)  # or int(home_team) based on your data
            month_int = int(month)
            day_int = int(day)

            vars_in_constraint = [x_h_d_l[home_team_str, month_int, day_int, h] for h in hours_list if (home_team_str, month_int, day_int, h) in unique_keys_x_list]

            model.addConstr(sum(vars_in_constraint) == 1, "daily_hour_sum_constraint_{}_{}_{}".format(home_team, month, day))




for hour in nb_by_hour["hour"]:
    print(hour, nb_by_hour[nb_by_hour['hour'] == hour]['nb_of_occurences'].values[0])
    sum_x_vars = sum(x_h_d_l[l, m, d, hour] for l, m, d, h in x_h_d_l.keys() if h == hour)
    model.addConstr(sum_x_vars == nb_by_hour[nb_by_hour['hour'] == hour]['nb_of_occurences'].values[0], f"hour_constraint_{hour}")
    
# Define a large M value
M = 1000

# Iterate over each unique combination of location, month, day, and hour
for l, month_days in team_month_day_hours.items():
    for m, days in month_days.items():
        for d, hours_list in days.items():
            for h in hours_list:
                sum_x_vars = sum(x_h_d_l[l2, m, d, h] for l2 in team_month_day_hours if (l2, m, d, h) in unique_keys_x_list)
            
                # Constraint 1: If two or more games are scheduled, overlapping must be 1
                model.addConstr(sum_x_vars <= 1 + M * o_d_l[l, m, d, h],
                                f"overlap_max_constraint_{l}_{m}_{d}_{h}")
                
                # Constraint 2: If less than two games are scheduled, overlapping must be 0
                model.addConstr(sum_x_vars >= 2 - M * (1 - o_d_l[l, m, d, h]),
                                f"overlap_min_constraint_{l}_{m}_{d}_{h}")
                
model.update()



In [ ]:
# Initialize the objective function as a linear expression because r_h_m_l are constants
objective = gp.LinExpr()

i = 0
# Add the sum of products of x_h_d_l and r_h_m_l for matching hour, location, and month

for x_key in x_h_d_l.keys():
    l, m, d, h = x_key  # x_key is a tuple (h, d, l)
    r_key = (h, m, l)
    if r_key in values_dict:  # values_dict contains fixed values for r_h_m_l
        i += 1
        objective.add(x_h_d_l[x_key] * values_dict[r_key] * 0.99995)
     
# Add the sum of products of o_d_l and c_l for matching location
for o_key in o_d_l.keys():
    
    l,d,m,h = o_key  # o_key is a tuple (d, l)
    if l in capacities_dict:  # capacities_dict contains the capacities corresponding to c_l
        i+=1
        objective.add(o_d_l[o_key] * capacities_dict[l] * 0.00005)


# Set the objective function in the model
model.update()
model.setObjective(objective, GRB.MINIMIZE)

In [ ]:
num_terms = objective.size()
print("Number of terms in the objective function:", num_terms)



In [ ]:
# Print all constraints safely, handling potential encoding errors
print("\nConstraints:")
constraints = model.getConstrs()
for constr in constraints:
    lhs_expression = model.getRow(constr)
    rhs_value = constr.RHS
    print(f"{constr.ConstrName}: {lhs_expression} = {rhs_value}")



In [ ]:
## Optimize the model
model.optimize()

In [ ]:
if model.status == GRB.OPTIMAL:
    # Retrieve the values of the variables
    for v in model.getVars():
        print(f'{v.varName}: {v.x}')

In [ ]:
# List to hold the data
data = []

# Check if the solution exists
if model.status == GRB.OPTIMAL:
    # Iterate over the variables
    for v in model.getVars():
        # Check if the variable value is 1
        if round(v.x) == 1:
            # Parse the variable name
            print(v.varName.split(','))
            team, month, day, hour = v.varName.split(',')
            # Add to the list
            if team[0] == 'x':
                team = team[2:]
                hour = hour[:-1]
                data.append([team, int(month), int(day), int(hour)])
        else:
            print("no")

    # Create a DataFrame
    df_res = pd.DataFrame(data, columns=['Team', 'Month', 'Day', 'Hour'])
else:
    print("No optimal solution found.")

In [ ]:
df_res

In [ ]:
df_res["optimal_schedule"] = 1

In [ ]:
df_res

In [ ]:
df

In [ ]:
merge_df = pd.merge(df, df_res, left_on=["Home Team", "hour", "month", "day"], right_on=["Team", "Hour", "Month", "Day"], how="inner")

In [ ]:
merge_df

In [ ]:
plot_overall_percentage_stress_category(merge_df,'Percentage of Different Stress Categories for optimal game features', 'Stress Category 26')

In [ ]:
plot_overall_percentage_stress_category(df[df["True_Game"] == 1],'Percentage of Different Stress Categories for non optimal game features', 'Stress Category 26')

In [ ]:
plot_overall_percentage_policies_category(merge_df,'Percentage of Different Policies Categories for Available Timeslots', "policies_rcp26")

In [ ]:
plot_overall_percentage_policies_category(df[df["True_Game"] == 1],'Percentage of Different Policies Categories for Available Timeslots', "policies_rcp26")

In [ ]:
plot_overall_percentage_policies_category(df,'Percentage of Different Policies Categories for Available Timeslots', "policies_rcp26")

## 8. Comparison Analysis

### 8.1 Comparison distribution of UTCI, between the 2 periods 2002-2022 / 2024-2043

In [ ]:
future_df = pd.read_csv("datasets/full_UTCI_comparison_2024_2044_corrected.csv")

In [ ]:
past_df = pd.read_csv("datasets/UTCI_cat.csv")

In [ ]:
# Finding the global minimum and maximum values across all columns
min_value = min(past_df['utci'].min(), future_df['UTCI_26'].min(), future_df['UTCI_45'].min(), future_df['UTCI_85'].min())
max_value = max(past_df['utci'].max(), future_df['UTCI_26'].max(), future_df['UTCI_45'].max(), future_df['UTCI_85'].max())

# Defining a common set of bins
bins = np.linspace(min_value, max_value, 20)  # 20 bins

# Setting up the plot
plt.figure(figsize=(10, 6))

# Plotting normalized histograms with the same bins
plt.hist(past_df['utci'], bins=bins, alpha=1, label='Past UTCI', color='blue', density=True)
plt.hist(future_df['UTCI_26'], bins=bins, alpha=0.5, label='Future UTCI_26', color='green', density=True)
plt.hist(future_df['UTCI_45'], bins=bins, alpha=0.5, label='Future UTCI_45', color='yellow', density=True)
plt.hist(future_df['UTCI_85'], bins=bins, alpha=0.5, label='Future UTCI_85', color='red', density=True)

# Removing the grid and setting a white background
plt.grid(False)
plt.gca().set_facecolor('white')

# Adding titles and labels
plt.title('Normalized Histogram of UTCI values with Common Bins')
plt.xlabel('UTCI Value')
plt.ylabel('Density')
plt.legend(loc='upper right')

# Save the plot
plt.savefig('plots_story/comparison_utci_values_histogram_percentage.png', dpi=300, bbox_inches='tight')

# Display the plot
plt.show()


In [ ]:
past_df_copy = past_df
future_df_copy = future_df
past_df  = past_df[past_df["True_Game"] == 1]
future_df  = future_df[future_df["True_Game"] == 1]

# Finding the global minimum and maximum values across all columns
min_value = min(past_df['utci'].min(), future_df['UTCI_26'].min(), future_df['UTCI_45'].min(), future_df['UTCI_85'].min())
max_value = max(past_df['utci'].max(), future_df['UTCI_26'].max(), future_df['UTCI_45'].max(), future_df['UTCI_85'].max())

# Defining a common set of bins
bins = np.linspace(min_value, max_value, 20)  # 20 bins

# Setting up the plot
plt.figure(figsize=(10, 6))

# Plotting normalized histograms with the same bins
plt.hist(past_df['utci'], bins=bins, alpha=1, label='Past UTCI', color='blue', density=True)
plt.hist(future_df['UTCI_26'], bins=bins, alpha=0.5, label='Future UTCI_26', color='green', density=True)
plt.hist(future_df['UTCI_45'], bins=bins, alpha=0.5, label='Future UTCI_45', color='yellow', density=True)
plt.hist(future_df['UTCI_85'], bins=bins, alpha=0.5, label='Future UTCI_85', color='red', density=True)

# Removing the grid and setting a white background
plt.grid(False)
plt.gca().set_facecolor('white')

# Adding titles and labels
plt.title('Normalized Histogram of UTCI values with Common Bins for True Games')
plt.xlabel('UTCI Value')
plt.ylabel('Density')
plt.legend(loc='upper right')

# Save the plot
plt.savefig('plots_story/comparison_utci_values_histogram_percentage_true_games.png', dpi=300, bbox_inches='tight')

# Display the plot
plt.show()


In [ ]:
len(future_df)

### 8.2 Comparison percentages categories between the 2 periods

In [ ]:

# Calculating percentage for each category in 'Stress Category' of past_df
past_category_counts = past_df['Stress Category'].value_counts(normalize=True) * 100

# Calculating percentages for each category in the relevant columns of future_df
future_category_26_counts = future_df['Stress Category 26'].value_counts(normalize=True) * 100
future_category_45_counts = future_df['Stress Category 45'].value_counts(normalize=True) * 100
future_category_85_counts = future_df['Stress Category 85'].value_counts(normalize=True) * 100

# Combining the results into a single dataframe
combined_df = pd.DataFrame({
    'Past': past_category_counts,
    'Future 26': future_category_26_counts,
    'Future 45': future_category_45_counts,
    'Future 85': future_category_85_counts
})

# Filling NaN values with 0 (in case some categories are not present in all datasets)
combined_df = combined_df.fillna(0)

for period in [26, 45, 85]:
    combined_df[f"difference_{period}"] = combined_df[f"Future {period}"] - combined_df["Past"]
combined_df.to_csv("temp/comparison_heat_stress_percentage.csv")
combined_df


## 9. Chi-square test

In [ ]:
future_df = pd.read_csv("datasets/full_UTCI_comparison_2024_2044_corrected.csv")
past_df = pd.read_csv("datasets/UTCI_cat.csv")

In [ ]:
past_df

### 9.1 chi square for difference in percenatges for the 3 scenarios

In [ ]:
data = future_df
# Check if the 'Stress Category' columns exist in the dataset
if 'Stress Category 26' in data.columns and 'Stress Category 45' in data.columns and 'Stress Category 85' in data.columns :
    # Calculate the frequency (as percentage) of each category for each stress category column
    freq_percent_26 = data['Stress Category 26'].value_counts(normalize=True) * 100
    freq_percent_45 = data['Stress Category 45'].value_counts(normalize=True) * 100
    freq_percent_85 = data['Stress Category 85'].value_counts(normalize=True) * 100
    
    # Combine these into a single DataFrame for comparison
    combined_percentages = pd.DataFrame({'Stress Category 26': freq_percent_26,
                                         'Stress Category 45': freq_percent_45,
                                         'Stress Category 85': freq_percent_85}).fillna(0)

else:
    "Stress Category columns not found in the dataset."


In [ ]:

# Calculate the raw count of each category for each stress category column
count_raw_26 = data['Stress Category 26'].value_counts()
count_raw_45 = data['Stress Category 45'].value_counts()
count_raw_85 = data['Stress Category 85'].value_counts()

# Create a contingency table for the raw counts
contingency_table_raw = pd.DataFrame([count_raw_26, count_raw_45, count_raw_85]).fillna(0)

# Chi-square test of independence on raw counts
chi2_raw, p_raw, dof_raw, expected_raw = chi2_contingency(contingency_table_raw)



v = 0
c = 0
for i in expected_raw:
    for j in i:
        c += 1
        if j > 5:
            v += 1
print("percentage expected value > 5", (v / c) * 100)
chi2_raw, p_raw, dof_raw, expected_raw, (expected_raw > 5)

In [ ]:
contingency_table_raw

In [ ]:
# Calculate the standardized residuals
residuals = (contingency_table_raw - expected_raw) / np.sqrt(expected_raw)

# Standardized residuals
standardized_residuals = residuals / np.sqrt(expected_raw)

standardized_residuals

### 9.2 chi square for differences in percentages between the 4 

In [ ]:
future_df

In [ ]:
data = future_df
data2 = past_df
# Check if the 'Stress Category' columns exist in the dataset
if 'Stress Category 26' in data.columns and 'Stress Category 45' in data.columns and 'Stress Category 85' in data.columns and 'Stress  Category' in data2.columns:
    # Calculate the frequency (as percentage) of each category for each stress category column
    freq_percent_26 = data['Stress Category 26'].value_counts(normalize=True) * 100
    freq_percent_45 = data['Stress Category 45'].value_counts(normalize=True) * 100
    freq_percent_85 = data['Stress Category 85'].value_counts(normalize=True) * 100
    freq_percent_past= data2['Stress Category'].value_counts(normalize=True) * 100
    
    # Combine these into a single DataFrame for comparison
    combined_percentages = pd.DataFrame({'Stress Category 26': freq_percent_26,
                                         'Stress Category 45': freq_percent_45,
                                         'Stress Category 85': freq_percent_85,
                                        'Stress Category Past': freq_percent_past}).fillna(0)

else:
    "Stress Category columns not found in the dataset."


In [ ]:
# Calculate the raw count of each category for each stress category column
count_raw_26 = data['Stress Category 26'].value_counts()
count_raw_45 = data['Stress Category 45'].value_counts()
count_raw_85 = data['Stress Category 85'].value_counts()
count_raw_past = data2['Stress Category'].value_counts()

# Create a contingency table for the raw counts
contingency_table_raw = pd.DataFrame([count_raw_past,count_raw_26, count_raw_45, count_raw_85]).fillna(0)

# Chi-square test of independence on raw counts
chi2_raw, p_raw, dof_raw, expected_raw = chi2_contingency(contingency_table_raw)



v = 0
c = 0

for i in expected_raw:
    for j in i:
        c += 1
        if j > 5:
            v += 1
print("percentage expected value > 5", (v / c) * 100)
chi2_raw, p_raw, dof_raw, expected_raw, (expected_raw > 5)

In [ ]:
# Calculate the standardized residuals
residuals = (contingency_table_raw - expected_raw) / np.sqrt(expected_raw)

# Standardized residuals
standardized_residuals = residuals / np.sqrt(expected_raw)

standardized_residuals

### 9.3 comparison between past and rcp2.6

In [ ]:
# Calculate the raw count of each category for each stress category column

# Create a contingency table for the raw counts
contingency_table_raw = pd.DataFrame([count_raw_past,count_raw_26]).fillna(0)

# Chi-square test of independence on raw counts
chi2_raw, p_raw, dof_raw, expected_raw = chi2_contingency(contingency_table_raw)



v = 0
c = 0
for i in expected_raw:
    for j in i:
        c += 1
        if j > 5:
            v += 1
print("percentage expected value > 5", (v / c) * 100)
chi2_raw, p_raw, dof_raw, expected_raw, (expected_raw > 5)

In [ ]:
# Calculate the standardized residuals
residuals = (contingency_table_raw - expected_raw) / np.sqrt(expected_raw)

# Standardized residuals
standardized_residuals = residuals / np.sqrt(expected_raw)

standardized_residuals

### 9.4 comparison between past and rcp4.5

In [ ]:
# Calculate the raw count of each category for each stress category column

# Create a contingency table for the raw counts
contingency_table_raw = pd.DataFrame([count_raw_past,count_raw_45]).fillna(0)

# Chi-square test of independence on raw counts
chi2_raw, p_raw, dof_raw, expected_raw = chi2_contingency(contingency_table_raw)



v = 0
c = 0

for i in expected_raw:
    for j in i:
        c += 1
        if j > 5:
            v += 1
print("percentage expected value > 5", (v / c) * 100)
chi2_raw, p_raw, dof_raw, expected_raw, (expected_raw > 5)

In [ ]:
# Calculate the standardized residuals
residuals = (contingency_table_raw - expected_raw) / np.sqrt(expected_raw)

# Standardized residuals
standardized_residuals = residuals / np.sqrt(expected_raw)

standardized_residuals

### 9.5 comparison between past and rcp8.5

In [ ]:
# Calculate the raw count of each category for each stress category column

# Create a contingency table for the raw counts
contingency_table_raw = pd.DataFrame([count_raw_past,count_raw_85]).fillna(0)

# Chi-square test of independence on raw counts
chi2_raw, p_raw, dof_raw, expected_raw = chi2_contingency(contingency_table_raw)



v = 0
c = 0

for i in expected_raw:
    for j in i:
        c += 1
        if j > 5:
            v += 1
print("percentage expected value > 5", (v / c) * 100)
chi2_raw, p_raw, dof_raw, expected_raw, (expected_raw > 5)

In [ ]:
# Calculate the standardized residuals
residuals = (contingency_table_raw - expected_raw) / np.sqrt(expected_raw)

# Standardized residuals
standardized_residuals = residuals / np.sqrt(expected_raw)

standardized_residuals

## 10. Bias Correction

In [ ]:
obs_filepath='datasets/UTCI.csv' #observed 
obs_df=pd.read_csv(obs_filepath)

hist_filepath='datasets/UTCI_2002_2023_proj_and_histo.csv' # calculated using models 
hist_df=pd.read_csv(hist_filepath)


futures={}
for scenario in [26,45,85]:
    future_filepath=f'UTCI_2024_2044_{scenario}.csv'
    futures[f'rcp{scenario}']=pd.read_csv(future_filepath)

In [ ]:
temp = obs_df
temp["dup"] = obs_df.duplicated(["Stadium", "timestamp"])
temp[temp["dup"] == True].sort_values(["Stadium", "timestamp"])

In [ ]:
hist_df.head()

In [ ]:
obs_df.duplicated().any()

In [ ]:
full_bc_historical_df=pd.DataFrame({})
full_bc_future_df=pd.DataFrame({})

for stadium in hist_df['Stadium'].unique():

    # Process historical
    stadium_df=obs_df[obs_df['Stadium']==stadium].sort_values('True_Game', ascending= False).drop_duplicates(['timestamp'])
    obs_time=stadium_df['timestamp'].apply(lambda x: pd.to_datetime(x))
    obs_utci=stadium_df['utci']-273.15

    stadium_df=hist_df[hist_df['Stadium']==stadium].drop_duplicates(['timestamp'])
    hist_time=stadium_df['timestamp'].apply(lambda x: pd.to_datetime(x))
    hist_utci=stadium_df['UTCI']

    observed_da = xr.DataArray(obs_utci, dims='time', coords={'time': obs_time},attrs={'units': 'degC'})
    model_da = xr.DataArray(hist_utci, dims='time', coords={'time': hist_time},attrs={'units': 'degC'})

    qdm_mapping=sdba.QuantileDeltaMapping.train(observed_da,model_da)
    
    corrected_model_da = qdm_mapping.adjust(model_da)

    bc_historical_df = corrected_model_da.rename('utci_bc').to_dataframe().reset_index()
    for column_to_copy in ['Home Team', 'True_Game','Stadium','latitude','longitude','Capacity','Division']:
        bc_historical_df[column_to_copy]=stadium_df.reset_index()[column_to_copy]
    
    full_bc_historical_df=pd.concat([full_bc_historical_df,bc_historical_df],ignore_index=True)

    bc_future_df=pd.DataFrame({})
    for scenario in [26,45,85]:
        future_df=futures[f'rcp{scenario}']
        stadium_df=future_df[future_df['Stadium']==stadium].drop_duplicates(['timestamp'])
        future_time=stadium_df['timestamp'].apply(lambda x: pd.to_datetime(x))
        future_utci=stadium_df['UTCI']

        if scenario==26:
            for column_to_copy in ['timestamp','Home Team', 'True_Game','Stadium','latitude','longitude','Capacity','Division']:
                bc_future_df[column_to_copy]=stadium_df.reset_index()[column_to_copy]
        
        model_da = xr.DataArray(future_utci, dims='time', coords={'time': future_time},attrs={'units': 'degC'})
        corrected_model_da = qdm_mapping.adjust(model_da)
        bc_future_df[f'utci_bc_rcp{scenario}'] = corrected_model_da.rename(f'utci_bc_rcp{scenario}').to_dataframe().reset_index()[f'utci_bc_rcp{scenario}']

          
        
    full_bc_future_df=pd.concat([full_bc_future_df,bc_future_df],ignore_index=True)



In [ ]:
bins = np.arange(-20,51,5)
fig,ax=plt.subplots()
plt.hist((obs_df['utci']-273.15),bins=bins, histtype='step', color='k', edgecolor='k',linewidth=2)
plt.hist(hist_df['UTCI'],bins=bins, histtype='step', color='grey', edgecolor='grey')
plt.hist(full_bc_historical_df['utci_bc'],bins=bins, histtype='step', color='orange', edgecolor='orange')


In [ ]:
colors_scen={
    26:'g',
    45:'r',
    85:'m'
}
fig,ax=plt.subplots()
plt.hist(full_bc_historical_df['utci_bc'],bins=bins, histtype='step', color='orange', edgecolor='orange')
for scenario in [26,45,85]:
    plt.hist(full_bc_future_df[f'utci_bc_rcp{scenario}'],bins=bins, histtype='step', color=colors_scen[scenario], edgecolor=colors_scen[scenario])



In [ ]:
full_bc_historical_df

In [ ]:
full_bc_future_df.to_csv("datasets/corrected_future_UTCI.csv")